# 4.3 ベクターと埋め込み

In [ ]:
!pip install -r requirements.txt

In [2]:
vector_01 = [0.1, -0.5]

In [2]:
import os

import boto3
import json
import numpy as np
from pprint import pprint

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import matplotlib_fontja

bedrock_runtime = boto3.client(service_name='bedrock-runtime')

## 4.3.1

In [ ]:
body = json.dumps({
    "texts": ["日本の首都は東京です。"],
    "input_type": "search_document"
})

response = bedrock_runtime.invoke_model(
    body=body,
    modelId="cohere.embed-multilingual-v3",
    accept="*/*",
    contentType="application/json"
)
response_body = json.loads(response.get("body").read())
response_embeddings = response_body["embeddings"]
# 最初の5次元だけ表示
pprint(response_embeddings[0][:5])
pprint("number of dimensions of vector: %d" % len(response_embeddings[0]))

## 4.3.2

In [15]:
def generate_embed_vector(texts: list[str], bedrock_runtime) -> np.ndarray:
    """与えられた文字列から埋め込みベクターを取得する
    """
    body = json.dumps({
        "texts": texts,
        "input_type": "search_document"
    })
    response = bedrock_runtime.invoke_model(
        body=body,
        modelId="cohere.embed-multilingual-v3",
        accept="*/*",
        contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    # 数値処理がしやすいように NumPy で扱う配列に変換します
    return np.array(response_body["embeddings"]) 

In [ ]:
words = [
    "犬", "猫", "dog", "cat", "ペット", 
    "車", "car", "自転車", "一輪車", "新幹線", "飛行機"]
embeddings = generate_embed_vector(words, bedrock_runtime)
tsne = TSNE(n_components=2, random_state=42, perplexity=4)
compressed_color_vectors = tsne.fit_transform(embeddings)

plt.scatter(
    compressed_color_vectors[:, 0],
    compressed_color_vectors[:, 1],
)
for i, label in enumerate(words):
    plt.annotate(
        label, (compressed_color_vectors[i, 0], compressed_color_vectors[i, 1])
    )
plt.grid()

os.makedirs("fig", exist_ok=True)
plt.savefig("fig/words_animals.png")
    

## 4.4 ベクターデータを使った検索システム例

In [14]:
import json
import logging

from botocore.exceptions import ClientError
from mypy_boto3_bedrock_runtime.client import BedrockRuntimeClient


def generate_message(
    bedrock_runtime: BedrockRuntimeClient,
    model_id: str, 
    system_prompt: str,
    messages: list[str], 
    max_tokens: int):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages
        }  
    )
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
   
    return response_body


def invoke_model(
    system_prompt: str, 
    user_input: str, 
    model_id: str = "anthropic.claude-3-haiku-20240307-v1:0",
    response_prefix: str = None
):
    """
    Claude Message API を利用するためのサンプル。
    """
    try:
        max_tokens = 1000
        # Prompt with user turn only.
        user_message =  {"role": "user", "content": user_input}
        messages = [user_message]
        if response_prefix:
            messages.append({"role": "assistant", "content": response_prefix}) 
        response = generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens)
        return response
    except ClientError as err:
        message=err.response["Error"]["Message"]
        logging.error("A client error occurred: %s", message)
        print("A client error occured: " +
            format(message))

In [4]:
def calculate_similarity_between_two_sentences(text1, text2, bedrock_runtime):
    embeddings = generate_embed_vector(
        [text1, text2],
        bedrock_runtime
    )
    cosine_similarity_result = cosine_similarity(embeddings[:1], embeddings[1:])
    euclidean_distance_result = np.linalg.norm(embeddings[1] - embeddings[0])
    print("text1 ", text1)
    print("text2 ", text2)
    print(f"cosine similarity: {cosine_similarity_result[0][0]:.2f}")
    print(f"euclidean distance result: {euclidean_distance_result:.2f}")

In [ ]:
calculate_similarity_between_two_sentences(
    "明日の天気は晴れです。",
    "快晴",
    bedrock_runtime
)
print("--------------------------------")
calculate_similarity_between_two_sentences(
    "明日の天気は晴れです。なので、私は犬と一緒に散歩に出かけようと思う。きっと気持ち良いだろう。",
    "快晴",
    bedrock_runtime
)

In [17]:
bedrock_description = (
"Amazon Bedrock は、種々の高性能な基盤モデル (FM) を提供し、生成 AI アプリケーションの構築に必要な幅広い機能を備えた"
"フルマネージドサービスです。セキュリティ、プライバシー、責任ある AI により開発を簡素化します。"
"Amazon Bedrock の包括的な機能を使用すると、さまざまな人気の FM を実験すること、"
"微調整や検索拡張生成 (RAG) などの手法を用いて、データを利用してそれらの FM をプライベートにカスタマイズすること、"
"旅行の予約や保険金請求の処理から広告キャンペーンの作成や在庫管理まで、複雑なビジネスタスクを実行するマネージドエージェントを"
"作成することができます。これらを実行するためにコードを記述する必要はありません。Amazon Bedrock はサーバーレスであるため、"
"インフラストラクチャを管理する必要がありません。また、使い慣れた AWS サービスを使用して、生成 AI 機能をアプリケーションに"
"安全に統合してデプロイできます。"
)

In [28]:
import re
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
fixed_length_splitter = CharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=3,
    separator="" # 固定長で区切るために separator を空文字に設定
)
fixed_length_splitter.split_text(bedrock_description)

In [ ]:
def split_by_period(text, separator="。"):
    """正規表現を使って句点のみで分割"""
    # 句点で分割し、句点も保持
    parts = re.split(r'({separator})'.format(separator=separator), text)
    chunks = []
    current_chunk = ""
    for part in parts:
        if part == separator:
            current_chunk += part
            if current_chunk.strip():
                chunks.append(current_chunk.strip())
            current_chunk = ""
        else:
            current_chunk += part
    # 最後のチャンクを追加（句点で終わらない場合）
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    return chunks

split_by_period(bedrock_description, separator="。")

In [10]:
def get_titan_embed(chunk: str) -> list[float]:
    """Amazon Titan Embeddings V2 を使用して埋め込みベクターを取得する
    """
    body = json.dumps({
        "inputText": chunk,
        "dimensions": dimension, # 1024, 512, 256 から選択が可能
        "normalize": True
    })

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId="amazon.titan-embed-text-v2:0",
        accept="*/*",
        contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())
    return response_body["embedding"]


chunks = [
    "A さんは 3 月 15 日に散歩をして、犬に会った。犬は飼い主とはぐれてしまい、一緒に飼い主を探した。",
    "B さんは 3 月 14 日に自転車でお出かけをして海まで行った。あいにくの曇り模様で綺麗な写真がとれずがっかりした。"
]
embeddings = []
dimension = 256 # Amazon Titan Embeddings V2 の場合 1024, 512, 256 から選択可能
for chunk in chunks:
    embedding = get_titan_embed(chunk)
    embeddings.append(embedding)

### 4.4.3

In [ ]:
import faiss
# IndexFlatL2 がユークリッド距離, IndexFlatIP が内積だがベクターが正規化されているのでコサイン類似度になる
# インデックスの作成
index = faiss.IndexFlatIP(dimension) # dimension は 256 次元
index.add(np.array(embeddings).astype("float32"))
print("追加されたデータ数: ", index.ntotal)

In [ ]:
query = "A さんが 3 月 15 日に誰と会いましたか？"
query_embedding = get_titan_embed(query)
# k は検索の際に
distance, hit_index = index.search(np.array([query_embedding]).astype("float32"), k=1)
context = chunks[hit_index[0][0]]

print("問い合わせ文: ", query)
print("検索結果: ", chunks[hit_index[0][0]])

rag_prompt = f"""
<instruction>
あなたはユーザーからの質問に答える AI エージェントです。ユーザーからの質問に正確に回答します。
<context></context>は、ユーザーからの質問に答えるにあたって関連すると思われる情報です。
</instruction>

<context>
{context}
</context>
ユーザーからの質問<query></query>を<background></background>の情報を参考にしながら答えてください。
<query>
{query}
</query>
ユーザーからの質問の返答のみを返してください。
"""

response_from_rag_system = invoke_model("", rag_prompt)
print("--------------------")
print("RAG システム全体からの応答: ")
pprint(response_from_rag_system["content"][0]["text"])

# 4.5

## 回答との意味的類似度

In [ ]:
question = "日本で2番目に高い山はどこですか？"
answer = "北岳になります。"
wrong_answer = "奥穂高岳です。"
ground_truth = "北岳です。"

answer_embed = generate_embed_vector([answer], bedrock_runtime)
wrong_answer_embed = generate_embed_vector([wrong_answer], bedrock_runtime)
ground_truth_embed = generate_embed_vector([ground_truth], bedrock_runtime)

answer_semantic_similarity = cosine_similarity(answer_embed, ground_truth_embed)
wrong_answer_semantic_similarity = cosine_similarity(wrong_answer_embed, ground_truth_embed)

print("good answer semantic similarity: ", answer_semantic_similarity[0][0])
print("wrong answer semantic similarity: ", wrong_answer_semantic_similarity[0][0])

In [ ]:
pprint(answer_embed)

## 回答の関連性

In [ ]:
import re 

question = "日本の首都はどこですか？"
answer = "日本の首都は東京です。東京は日本の首都であり、人口約1,400万人を擁する世界有数の大都市です。東京都心部には皇居、国会議事堂、主要オフィスビルなどが集中しており、日本の政治・経済・文化の中心地となっています。"

question_generate_prompt = f"""
回答<answer></answer>から想定される質問内容を<questions></questions>タグの中で 3 つ生成してください。
<answer>
{answer}
</answer>
出力フォーマットは下記の通りとします。
<questions>
   <question>{{ここに回答内容を記す}}</question>
   <question>{{ここに回答内容を記す}}</question>
</questions>
"""
response = invoke_model("", question_generate_prompt)
response_text = response["content"][0]["text"]

pattern = r"<question>(.*?)</question>"
generated_questions = re.findall(pattern, response_text, re.DOTALL)
print("Generated questions: ", generated_questions)
# ['日本の首都はどこですか?', '東京はどのような都市ですか?', '東京には何が集中していますか?']


generated_questions_embeddings = generate_embed_vector(generated_questions, bedrock_runtime)
questions_embedding = generate_embed_vector([question], bedrock_runtime)
cosine_similarities = cosine_similarity(questions_embedding, generated_questions_embeddings)
print(cosine_similarities)
print("Answer Relevance Score is: ", cosine_similarities.mean())

## Faithfulness

In [43]:
# context 情報は新幹線の wiki から取得  
# https://ja.wikipedia.org/wiki/%E6%96%B0%E5%B9%B9%E7%B7%9A
context = """
新幹線（しんかんせん、英: Shinkansen）は、主たる区間を列車が時速200キロメートル (km/h) 以上の高速度で走行できる日本の幹線鉄道[1]。

「高速鉄道」のコンセプトを世界に広めた存在であり、速度・輸送力（座席数）・安全性において世界の高速鉄道の先駆けとなる存在である[2]。
概要

日本国有鉄道（国鉄）が1964年（昭和39年）10月1日に東海道本線の線路容量逼迫（ひっぱく）対策として、東京駅 - 新大阪駅間の線増区間として開業した東海道新幹線を端緒とする。その後国鉄時代には山陽本線の線増区間として建設された山陽新幹線の開業を経て、全国新幹線鉄道整備法に基づく基本計画を根拠として東北・上越の各新幹線が開業した。国鉄分割民営化により国鉄の事業がJRに移行した後は、北陸[注 2]・九州（鹿児島ルート）・北海道・西九州（九州〈西九州ルート〉の一部）の4路線が開業し、加えて従来のJR線（在来線）と新幹線とで直通運転を行うミニ新幹線（法令上は標準軌の在来線。詳細後述）として山形・秋田の2路線が開業しており、新幹線網の拡大は半世紀にわたって続けられている。2024年（令和6年）時点でも北海道・北陸・中央の各新幹線が建設中である。

2023年（令和5年）の時点で、フル規格8路線（合計2,830 km）とミニ新幹線2路線（合計276 km）が営業中で、2015年度（平成27年度）の年間利用者数は3億6000万人に上る[3]。

高速度での都市間輸送を前提としており、在来線と異なる規格（軌間・線形・架線電圧など）を採用するなど、様々な技術的特徴がある（#主要技術参照）。走行する車両も空力や低騒音に配慮した、流線形の外形を採用し高出力のモーターを搭載した専用の電車（新幹線車両）が用いられている。こうした技術投入の結果、定時性が極めて高く、年間13万本以上の列車が運転される東海道新幹線でも平均遅延時間は24秒に留まる（2016年度）。また50年以上にわたる新幹線の歴史の中で、車両や線路の施設や設備の異常、運行側の不手際などに起因する乗客などの死亡事故は一度も発生していない（#安全性参照）。
建設

新幹線建設の計画（建設を開始すべき新幹線鉄道の路線を定める基本計画）は全国新幹線鉄道整備法第4条の規定に基づき国土交通大臣が決定するものと定められている[4][5]。

全国新幹線鉄道整備法施行以前の新幹線路線の建設工事については、東海道新幹線・山陽新幹線は事業主体である日本国有鉄道（国鉄）が建設主体でもあった。全国新幹線鉄道整備法施行後は同法第6条の規定に基づき国土交通大臣が営業主体および建設主体を指名することになっており、東北新幹線は国鉄が建設主体であった[6]が、上越新幹線は日本鉄道建設公団が建設主体となった[6]。国鉄分割民営化後の東北・上越新幹線東京延伸の際は、当時新幹線の地上設備を一括して所有していた[7]新幹線鉄道保有機構が建設主体であった[8]。さらにその後に建設された北陸新幹線や九州新幹線などのいわゆる整備新幹線については、日本鉄道建設公団および新幹線鉄道保有機構の権利を継承した鉄道建設・運輸施設整備支援機構が建設主体となっている。なお、超電導リニア方式を採用する中央新幹線については東海旅客鉄道（JR東海）が建設主体に指名された。
運営

運営は1964年（昭和39年）の開業から1987年（昭和62年）の国鉄分割民営化までは国鉄が行っており、国鉄の事業がJR各社に譲渡された際から、東北・上越新幹線は東日本旅客鉄道（JR東日本）が、東海道新幹線は東海旅客鉄道（JR東海）が、山陽新幹線は西日本旅客鉄道（JR西日本）が運営を行っている。

JR移行後に開業した整備新幹線では、北陸新幹線をJR東日本とJR西日本が、九州・西九州新幹線を九州旅客鉄道（JR九州）が、北海道新幹線を北海道旅客鉄道（JR北海道）が運営を行っている。なお、三島会社はJR発足当時いずれも新幹線を運営していなかったが、整備新幹線の開業に伴い、JR九州は2004年（平成16年）以降、JR北海道は2016年（平成28年）以降、新幹線を運営している。JR旅客6社のうち四国旅客鉄道（JR四国）は新幹線の運営に携わったことがない。
法的な位置づけ

冒頭で述べたとおり、全国新幹線鉄道整備法では、新幹線鉄道を「その主たる区間を列車が200キロメートル毎時（以降km/hと記す）以上の高速度で走行できる幹線鉄道」（第2条）と定義している[9]。「その主たる区間」であるから、局所的に200 km/h未満の速度でしか走行できない区間が存在しても新幹線鉄道である。新幹線を法律で定義しているのは、在来線とは異なる運転規則や構造規則（いずれも省令）が必要なためである。

列車の運行を妨げる行為に対しては、一般の鉄道でも鉄道営業法や刑法などに規定があるが[10]、それに加えて、新幹線鉄道における列車運行の安全を妨げる行為の処罰に関する特例法（新幹線特例法）などによって、より厳しい法的措置が定められている[10]。

なお、一般にミニ新幹線として称され、時刻表にも新幹線（の一部）として記載されている山形新幹線・秋田新幹線は、在来線を改軌して対応する一部の新幹線列車が走行できるようにしたものであり、目視による安全確認を要する信号機とATSの組み合わせによる列車防護、踏切道の存在など、新幹線鉄道構造規則に準じた構造（後述の「路線・軌道設備」を参照）とはなっておらず[5]、全国新幹線鉄道整備法上の新幹線鉄道には含まず、在来線の扱いとなっている（後述の「在来線への直通」並びに両路線の記事を参照）。
最高速度

これまでに建設された新幹線は、ミニ新幹線を除いて、1964年（昭和39年）に開業した東海道新幹線から全て設計最高速度260 km/hで建設されている[11]が、開業時から260 km/h運転を実施するようになったのは1997年（平成9年）10月1日開業の北陸新幹線（当時の通称は長野新幹線）からである。

現在の最高速度は320km/hとなっている。
日本の新幹線の営業最高速度 開業年月日 	路線 	全幹法
の分類 	通称 	最高速度
開業時 	2023年現在
1964年（昭和39年）10月1日 	東海道新幹線 	新幹線 	新幹線 	210 km/h 	285 km/h
1972年（昭和47年）3月15日 	山陽新幹線（岡山駅以東） 	300 km/h
1975年（昭和50年）3月10日 	山陽新幹線（岡山駅以西）
1982年（昭和57年）6月23日 	東北新幹線（盛岡駅以南） 	320 km/h（宇都宮駅 - 盛岡駅間）
1982年（昭和57年）11月15日 	上越新幹線 	275 km/h
1992年（平成4年）7月1日 	山形新幹線 	在来線 	ミニ新幹線 	130 km/h 	130 km/h
1997年（平成9年）3月22日 	秋田新幹線
1997年（平成9年）10月1日 	北陸新幹線（長野駅以南）[注 2] 	新幹線 	整備新幹線 	260 km/h 	260 km/h
2002年（平成14年）12月1日 	東北新幹線（盛岡駅以北）
2004年（平成16年）3月13日 	九州新幹線
2015年（平成27年）3月14日 	北陸新幹線（長野駅以北）
2016年（平成28年）3月26日 	北海道新幹線
2022年（令和4年）9月23日 	西九州新幹線
2024年（令和6年）3月16日 	北陸新幹線（金沢駅以西）
2027年（令和9年）予定 	中央新幹線 	リニア新幹線 	（建設中） 	505 km/h[12] 	
詳細な路線一覧は「路線」節を参照
呼称

「新幹線」という呼称は、かつての鉄道省が東京と下関を結ぶ高速鉄道計画に対して、遅くとも1939年（昭和14年）の時点で用いていた用語である[13]（ただし、この計画については当時の世相を踏まえて「弾丸列車」と呼ばれることの方が多かった）。さらに以前の大正期には「新しい幹線交通」を指す用語として「新幹線」の用語が用いられていたという[13]。

法律上、最初に「新幹線鉄道」の語が現れるのは、昭和39年6月22日法律第111号「東海道新幹線鉄道における列車運行の安全を妨げる行為の処罰に関する特例法」（現「新幹線鉄道における列車運行の安全を妨げる行為の処罰に関する特例法」）である。東海道新幹線は在来線である東海道本線の線増として建設されたために「東海道新幹線鉄道」とされた。

『新幹線』という名称は、東海旅客鉄道・東日本旅客鉄道・西日本旅客鉄道の3社名義にて、複数の分野で商標登録されている。例えば「鉄道による輸送」では第3066558号である。

日本国内の駅内の案内板等の英語表記では、路線を指す場合は「しんかんせん」のローマ字表記である Shinkansen を使用し、列車名（便名）を表す場合は、各駅に停車する列車も含め、かつて「ひかり」の種別として用いられていた「超特急」の直訳である Superexpress を愛称名の後ろに付けて「NOZOMI Superexpress」などと表現している。これは特急（特別急行列車、Limited express）と急行列車（Express）を異なる種別として認識し、さらに新幹線を（通常の）特急とも区別しようとする日本独特の表現とも考えられる[14]。

日本国外において新幹線について言及する際は「Shinkansen」と表記される一方で、戦前からの計画名称である「弾丸列車」の直訳である「Bullet train」の表現を用いることがある。例えば、1975年（昭和50年）に日本で公開され、翌年アメリカで公開された映画『新幹線大爆破』の英題の1つも "The Bullet Train" であり、2022年公開の新幹線を舞台にしたブラッド・ピット主演の映画タイトルも「ブレット・トレイン」(Bullet Train) である）。
主要技術

新幹線鉄道は、その大部分の区間において200 km/hを超える速度で運行するため、在来線鉄道とは異なったさまざまな技術が用いられている。速度のみならず、乗り心地や安全面でも世界的に見ても非常に高い水準が確保されている。
路線・軌道設備
標準軌 (1,435 mm)。
東海道新幹線で最小の曲率半径500m
（品川 - 新横浜間・武蔵小杉駅付近）
新幹線の高架橋
（上越新幹線回送線・新潟市）
踏切は設置せず、立体交差と防護柵で安全運行を図る
（東海道新幹線・名古屋市緑区）
プラットホームの安全柵
（東海道新幹線・東京駅）

    路線は、在来線と別ルートで新規に建設した線路設備を用いる。設備の構造については省令の「新幹線鉄道構造規則」に規定されている。在来線を改良したミニ新幹線と区別するため、「フル規格」とも呼ばれる。
        軌間は標準軌 (1,435 mm) を用いている。ただし標準軌が「新幹線」の法的な条件というわけではなく[15]、軌間に狭軌 (1,067 mm) を用いつつ高速走行を可能とした「新幹線」もありえる[15]、こうした方式の新幹線を新幹線鉄道規格新線（スーパー特急）し、この方式で建設に着手された路線もある（ただしいずれも建設途中で標準軌規格に変更されており、スーパー特急方式で開業した新幹線路線は存在しない）。
        カーブにおける曲率半径を大きくし、できる限り直線を確保する。本線区間における最小曲率半径は東海道新幹線が2,500 m[16][17]、山陽新幹線以降に建設された各線は4,000 m[16][17]となっている。ただし、用地や地形の関係から急曲線とならざるを得ない区間では、その区間の列車速度により曲率半径400 mまで許容されている。さらに推定脱線係数比が一定以上か、脱線防止ガードを設置することで200 m以上の曲率半径をとることもできる。東海道新幹線の東京 - 新横浜間や東北新幹線の東京 - 大宮間のような都心部区間は、曲率半径が400 mから2,000 m程度の急曲線が含まれている。
        勾配は高速走行の妨げになることから最急勾配を15 ‰までとするが、延長2.5 km以内に限り18 ‰・2.0 km以内に限り20 ‰とする。用地や地形の関係から規定以上の勾配を必要とする区間では特別認可の形で設置されており、東北新幹線東京 - 大宮間では25 ‰、北陸新幹線では30 ‰、九州新幹線鹿児島ルートでは35 ‰の勾配が含まれている。
    事故防止のため以下の設計を行う。
        自動車との衝突事故を防ぐため、踏切を一切設けない。
        線路内に一般の人が立ち入れないようにする。前項も含めた対策として全線立体交差とする。また、列車の運行妨害等に対しては法律面でも「新幹線特例法」によって在来線より厳しい罰則を定めている。
        通過列車との接触など人身事故を防ぐため、プラットホームに可動ゲート付きの安全柵（以下、ホームドア）を設ける（例：新横浜駅や新神戸駅など）か、通過線と待避線を分ける（例：静岡駅、福島駅など）。ただし、通過列車の通過速度が低い駅（例: 上野駅）には安全柵のみ設けられているか、安全柵が設けられていない場合もある。また、東海道新幹線・山陽新幹線の東京駅や名古屋駅、京都駅、新大阪駅、岡山駅、広島駅、小倉駅、博多駅など、全列車が停車する駅には当初柵などは設けられてはいなかったが、後に安全柵ならびに一部の駅でホームドアが設けられた。また、東海道新幹線では、静岡駅や浜松駅など、通過線と待避線が分かれていながら安全柵が設置されている駅もある。九州新幹線鹿児島ルートでは全列車が停車する熊本駅、鹿児島中央駅や通過列車が使用しない副本線のホームも含め、開業当初から全ての駅の全てのホームにホームドアが設置されている。東北新幹線の新青森駅でも全てのホームにホームドアが設置されている。
        東北新幹線の盛岡駅以南では、ミニ新幹線で使われている在来線規格の車両とフル規格対応のホームとの間に隙間が生じるため、駅停車時にホーム側へ張り出すステップを車両に設置したり、ホームにドア付近以外での転落を防止するための安全柵を設けたりする対策がなされている。
    乗り心地や安全性の向上、騒音対策などから、レールや分岐器（ポイント）にもさまざまな工夫が施されている。
        レールは、継ぎ目の数を減らしたロングレールを使用。東北新幹線のいわて沼宮内駅 - 八戸駅間には、国内最長の延長約60.4 kmにわたる「スーパーロングレール」が用いられている。
        分岐器（ポイント）は、通過時の振動が少ない弾性分岐器と、レール交差部の欠線部を埋めるノーズ可動クロッシングを使用。また、高崎駅北方にある上越新幹線と北陸新幹線との分岐には、分岐側を160 km/hで通過できる国内最高水準かつ最長の分岐器が設置されている。
    新幹線の駅間距離は、中距離・長距離輸送を主とすることから、原則として在来線より長く取られている（30 - 40 km程度）。
    高速運転で駅間距離が長く、より迅速で的確に情報伝達を行うため、列車無線を開業当初から採用している。

信号システム

    地上装置と車上装置からなる自動列車制御装置 (ATC) と列車集中制御装置 (CTC) を備えている[18]。ATCは、地上装置に沿線の20-30kmの間隔に信号機器室を設けて、そこから信号ケーブルを介して軌道回路に信号電流を流し、車上装置にそれを受電器で受信して運転室内に運行指示（許容速度）が表示され[注 3][18]、その速度を超えれば自動的にブレーキが作用するもので[18]。自動ブレーキが作動するのは営業最高速度やカーブなどの速度制限を超えようとした時、先行列車に接近した時、駅に停車するために減速する時などである。駅停車時は15 - 75 km/h以下の低速時になると手動でブレーキ弁を操作して列車を停止位置目標に停止させるが、目標の少し先で停止するようなパターンが作成されるか（TASCではない）、または目標の先方50 mで強制的に非常ブレーキが掛かる区間になっており、過走を防止している。これは地上の信号機を車上から目視確認して運転することは（気象状況によっては）困難となる[18]ほどの高速運転を行うためである。また故障による影響を最小限とするため、同じ機能を持つシステムを3系統備えており、そのうち1系統が故障しても3者の多数決の原理で残った2系統で正常に作動し運転を続行できるようになっている[19]。
    CTCは、列車の位置と列車番号の表示や各駅の分岐器を運転指令所で一括管理と制御を行うもので、これですべての列車の運行状況を一括管理している。現在では列車運行管理システム (PTC) も導入されており、通常の分岐器操作や信号制御、駅自動放送から車両の管理整備、輸送障害時の復旧ダイヤの作成に至るまで、あらゆる業務がコンピュータによって高度にシステム化されている。

電源方式

    単相交流25,000 Vで電力を供給する。饋電（きでん）方式については、東海道新幹線開業当初はBT方式であったが、現在では他の新幹線と共にAT方式に統一された。電源周波数は以下の通り。
        東海道新幹線では60 Hzに統一して給電している。静岡県の富士川を境に50 Hz（東側）と60 Hz（西側）の電源周波数区分が異なるが、50/60 Hz を共用とした場合 約 2.7t の車体重量増加が見込まれ将来的な経済性なども考慮し[20]、当初から山陽方面への延長を構想していたため全線で60 Hzに統一し、車両側の特高圧機器の簡素化を図っている。なお、電源周波数区分50 Hzの地域では周波数変換所が設けられ、新幹線電源用に60Hzに変換している。
        北陸新幹線は軽井沢駅 - 佐久平駅、上越妙高駅 - 糸魚川駅と糸魚川駅 - 黒部宇奈月温泉駅の計3か所に50/60 Hzの切り替えセクションが存在[21]し、車両側も50/60 Hzの双方に対応している。
        上記以外の各新幹線は沿線地域と同じ電源周波数で、山陽・九州は60 Hz、東北・上越・北海道は50 Hz。
    いずれの電気方式においても、変電所間での位相（北陸新幹線においては周波数）の相違を解決する必要があるが、高速を維持するため連続力行運転を行うことから、変電所の饋電区間の境界は、在来線のようにデッドセクション（アーク発生防止のため惰行で通過する）ではなく、地上切替方式を採用している。切替区間はエアーセクションで区分され、その前後の変電所の双方から饋電でき、最初は進入側の変電所から饋電し、列車が切替区間に入ったことを検知すると進出側の変電所からの饋電に切り替える。この間はおよそ0.5秒程度であり、乗客が切替を感知することはほとんどない。
    送電側の系統障害を避ける必要から[注 4]、スコット結線変圧器や変形ウッドブリッジ結線変圧器、ルーフ・デルタ結線変圧器を用いて三相交流から90度位相の異なる2組の単相交流が作られ[20]、それぞれ上り線と下り線に給電されている[22][23]。変電所の設置間隔は約 20 km 毎である[20]。

車両技術
詳細は「新幹線車両」を参照
東海道・山陽新幹線歴代車両。
左から700系、300系、0系

新幹線では、動力を編成各車両に分散させる「動力分散方式」が採用されている。動力分散方式を採用することにより、電車方式と同様の、加減速性能の向上・軽量化・軌道への負荷軽減といった利点が追求されている。 また高速走行を行うため、列車編成内における電動車（動力車）の比率（MT比）が極力大きくされている。ブレーキは主電動機の発電抵抗を利用する電気ブレーキと、空気圧動作の摩擦による基礎ブレーキを併用するが、高速域からの減速には主に電気ブレーキが使用される。こうすることによって制輪子の磨耗を抑え、交換周期を延ばすことができる。

また、車両には気密構造が採用されている。高速運転時にトンネルに進入するなどの気圧変動による居住性の低下を防ぐためである。また、0系や100系など国鉄時代の東海道・山陽新幹線車両では車体の素材に普通鋼が使われていたためやや重かったが、東北・上越新幹線用の200系からは耐雪装備による重量増加を抑えるためアルミニウムが用いられて軽量化が図られた。国鉄民営化後に開発された新幹線車両はアルミニウム車体が一般化、さらにアルミ材の加工手法の発達により、製作費のコストダウンとさらなる軽量化の両立が図られた。この結果、国鉄時代に開発された初期新幹線車両より著しく軽量化されている。

一方で、JR発足以降積極的に行われた高速化に伴い、走行中のパンタグラフと架線の接触や風切り音による騒音の発生や、接触部の著しい消耗などが問題とされた。このため、0系では2両おきに付いていたパンタグラフが300系では8両毎に1つに減ったほか、500系では翼型と呼ばれるT字型の特殊な集電装置が設置されるなど改良され、騒音を抑えながら集電効率を向上させた。また、パンタグラフに流線型の突起物を取り付けるなどの改良も加えられた。その他、高速でのトンネルの突入時のトンネル内部の急激な気圧変化による騒音（トンネル微気圧波）の発生を抑えるための、走行時の空気の流動性やトンネル進入時の面積変化率を考えた先端車両の開発などが行われているため、初期の0系に比べ先頭車先端部が長く伸ばされるとともに、通常の電車とは著しく異なった形態（鋭い流線型やカモノハシのような形）を呈する傾向にある。
列車防護装置

高速走行を行うため、在来線と同じ信号炎管や軌道短絡器による列車防護（他の列車を停止させること）では他の列車が停止しきれない可能性が高まる。そのため、緊急時に他の列車を迅速に停止させられるように在来線とは異なる列車防護の方式が採られている。

    車両側には保護接地スイッチ (EGS) が装備され、緊急時には乗務員が運転台の「保護接地入スイッチ」を押すことにより、他の列車を自動的に停止させることができる。
    線路側には列車防護スイッチが、本線上には250 m間隔、ホーム上には50 m間隔で設置され、これを押すことでATC回路を停止信号にすることができる。
    列車防護無線装置は車両には受信機のみが装備され、発信器は保線作業中に線路を支障させた場合、保安方式変更などでATCを使用していない列車を停止するため保線係員が携帯している。

在来線との直通

前述のとおり、新幹線は軌間や電圧、運行管理システムなど在来線と異なる技術基準が多く採用されており、基本的には他国の高速鉄道で行われているような在来線との直通運転はできない。しかし、新幹線列車を在来線に直通運転させる、あるいは在来線列車を新幹線規格の路線に走行させて利便性を確保しようとする研究が行われている。

ミニ新幹線
    現在唯一実用化されている技術で、全国新幹線鉄道整備法附則第6項第2号に定めるところの「新幹線鉄道直通線」、すなわち既存の鉄道路線に敷設された線路が新幹線と接続し、かつ新幹線の列車が省令で定める速度で走行できる構造を有する路線であり、在来線の線路を新幹線のものと同じ標準軌に改軌改良し、在来線の車両限界に合わせ、在来線の安全基準にも合致した車両で新幹線との直通運転を行う方式。山形新幹線の福島駅以北並びに秋田新幹線の盛岡駅以西がこれに該当する。
    新幹線区間と標準軌在来線を直通する特急列車を新幹線直行特急という。
    この方式で標準軌に改軌改良された在来線区間は、以下のような特徴がある。

        法律や設備などの上では新幹線ではなく在来線である。ただし、営業上・案内上では「山形新幹線」「秋田新幹線」といった「新幹線」の呼称が用いられる。営業戦略上と地元への誘致効果がその理由である[24]。
        最高速度は130 km/hに制限されている[25]。
        完全立体交差化は行わず、踏切数を削減すると共に保安設備を強化している。
        ミニ新幹線化された区間の全区間が改軌前より50 Hz・20,000 V交流電化された区間であったため、改軌後もこれをそのまま採用し、電圧は20,000 Vのままである。直通車両は複数電源対応としている。この場合の異電圧区間の接続はデッドセクションとなっている。

フリーゲージトレイン
    車両側の台車を1067 mm軌間と1435 mm軌間の両方に対応させる方式。
    鉄道総合技術研究所（JR総研）により開発が進められているが、高速運転を前提とした導入には技術的障壁が多いことが明らかになっており（当該記事参照）、導入に至っていない。
新幹線鉄道規格新線（スーパー特急）
    全国新幹線鉄道整備法附則第6項第1号に定められた方式で、新幹線規格の路盤上に在来線と同じ狭軌（1067mm軌間）の軌道を敷設する方式。200 km/hでの走行が可能とされ、在来線との直通が容易と考えられる。
    これまでいくつかの整備新幹線がこの方式で着工されたが、建設途中ですべて標準軌（フル規格）に変更されて建設されたため、現段階においてはこの方式を採用した路線は存在しない。

なお、北海道新幹線のうち青函トンネルとその前後区間は、フル規格の新幹線路線に狭軌を併設した三線軌条とした上で、新幹線規格の電圧・信号システムに対応した在来線車両（EH800形機関車など）が走行できる「在来線（海峡線）との共用区間」として取り扱っている。
路線
新幹線の路線図

まず1964年に東海道新幹線が開業し、これを延長する形で山陽新幹線の工事も始まり、1975年に博多駅まで全線開業した。

そして1970年には全国新幹線鉄道整備法が定められた。これによりまず東北・上越・成田の各新幹線の整備計画が決定し、続いて北海道新幹線、東北新幹線（ 盛岡市- 青森市間）、北陸新幹線、九州新幹線鹿児島ルート、同長崎ルート（西九州ルート）の5線の整備計画も決定された（整備新幹線）。

整備新幹線以前に計画された路線は、計画が失効した成田新幹線を除き開業しており、整備新幹線の一部も開業している。しかしその一方で、基本計画が定められたまま着工の目処が全く立っていない路線も存在する。

東海道新幹線と山陽新幹線を併せて「東海道・山陽新幹線」、東北新幹線と上越新幹線を併せて「東北・上越新幹線」と呼ぶことがある。東海道・山陽新幹線は国鉄時代は一体的な運用がなされており、民営化後も多くの列車の相互直通運転が行われているため一括して扱われることが多い。1982年に東北・上越新幹線が開業するまでは単に「新幹線」と呼ばれることもあった。東北・上越新幹線は1982年に相前後して開業した東側のフル規格新幹線で、東海道・山陽新幹線のような一体的な運用はないが、一部の区間を共用するほか、車両やATCなどの運行システムが共通である。

このほか、東海道・山陽新幹線にならって相互直通運転がなされている新幹線同士を総称し、「東海道・山陽・九州新幹線」、「山陽・九州新幹線」、「東北・北海道新幹線」、「北海道・東北新幹線[26]」と呼ぶことがある。
営業中の路線
標準軌新線（フル規格）

以下の8路線が開業している。北陸新幹線・北海道新幹線・西九州新幹線は一部分のみの開業である。
名称 	起点 	終点 	営業キロ 	実キロ 	駅数 	開業年月日 	運営会社
北海道新幹線[* 1] 	新青森駅 	新函館北斗駅 	148.8 km 	148.8 km 	4 	2016年3月26日：新青森駅 ‐ 新函館北斗駅 	北海道旅客鉄道（JR北海道）
東北新幹線 	東京駅 	新青森駅 	713.7 km 	674.9 km 	23 	

    1982年6月23日：大宮駅 - 盛岡駅
    1985年3月14日：上野駅 - 大宮駅
    1991年6月20日：東京駅 - 上野駅
    2002年12月1日：盛岡駅 - 八戸駅
    2010年12月4日：八戸駅 - 新青森駅

	東日本旅客鉄道（JR東日本）
上越新幹線 	大宮駅 	新潟駅 	303.6 km 	269.5 km 	10 	1982年11月15日 	東日本旅客鉄道（JR東日本）
北陸新幹線 	高崎駅 	上越妙高駅 	176.9 km 	176.9 km 	8 	

    1997年10月1日：高崎駅 - 長野駅
    2015年3月14日：長野駅 - 上越妙高駅

	東日本旅客鉄道（JR東日本）
上越妙高駅 	敦賀駅 	293.8 km 	293.8 km 	12 	

    2015年3月14日：上越妙高駅 - 金沢駅
    2024年3月16日：金沢駅 - 敦賀駅

	西日本旅客鉄道（JR西日本）
東海道新幹線 	東京駅 	新大阪駅 	552.6 km 	515.4 km 	17 	1964年10月1日 	東海旅客鉄道（JR東海）
山陽新幹線 	新大阪駅 	博多駅 	644.0 km[* 2] 	553.7 km 	19 	

    1972年3月15日：新大阪駅 - 岡山駅
    1975年3月10日：岡山駅 - 博多駅

	西日本旅客鉄道（JR西日本）
九州新幹線[* 3] 	博多駅 	鹿児島中央駅 	288.9 km 	256.8 km 	12 	

    2004年3月13日：新八代駅 - 鹿児島中央駅
    2011年3月12日：博多駅 - 新八代駅

	九州旅客鉄道（JR九州）
西九州新幹線[* 4] 	武雄温泉駅 	長崎駅 	69.6 km 	66.0 km 	5 	2022年9月23日：武雄温泉駅 ‐ 長崎駅 	九州旅客鉄道（JR九州）
計 			3,066.7 km 	2,830.6 km 	97 		5社

    ^ 北海道新幹線の新中小国信号場 - 木古内駅間（82.0 km）は海峡線と共用。
    ^ 山陽新幹線については、乗車券の有効期間の計算に使う岩徳線経由の営業キロでは618.5 km、運賃・料金計算に使われる同線経由の運賃計算キロは622.3 km。
    ^ 整備新幹線としては九州新幹線（鹿児島ルート）と称されていたが、開業後は単に「九州新幹線」として営業しており、時刻表や駅などに「鹿児島ルート」とは表記されない。
    ^ 整備新幹線としては九州新幹線（西九州ルート）や九州新幹線（長崎ルート）と称されていたが、開業後は「西九州新幹線」として営業しており、時刻表や駅などに「西九州ルート」や「長崎ルート」とは表記されない。

JR東日本とJR北海道は東北新幹線と北海道新幹線で、JR東日本とJR西日本は北陸新幹線で、JR東海とJR西日本は東海道新幹線と山陽新幹線で、JR西日本とJR九州は山陽新幹線と九州新幹線で、それぞれ相互直通運転を行っている。以前、山形新幹線用・秋田新幹線用の車両の一部は保有会社からの貸出であったが、現在はすべてJR東日本の所有する車両で運行されている。

東京駅では東海道新幹線と東北新幹線の線路が接続されていないため、博多や新大阪から新函館北斗まで（その逆も）直通列車で行くことはできず、東京駅での乗り換えが必要となる。国鉄時代の利用状況の調査で東京都内を通過する需要が非常に小さいということは判明していたが、当時博多・札幌開業の際には夜行列車の運転も計画されていた。そのため、全列車が東京駅での折り返すのであれば、ホームの容量は大幅に不足するとみられていた。そこで、両線を直通運転として東海道の列車は田端基地、東北・上越の列車は品川基地での折り返しとすることとして建設計画が立てられた。あわせて、田端以北に異周波デッドセクションを設けることが計画され、直通運転の試験車両として961形も製造された。実際に、東京駅の東海道新幹線14・15番線ホームは直通を想定して作られたため、ホームが東北新幹線側にカーブしている。それでも、東京駅のホーム容量が不足する状態となったときは、上越新幹線を大宮駅から新宿駅へ分岐させる構想としていた[27]。しかし、ダイヤの乱れが相互に波及し運転管理面に多くの問題が予想されること、周波数が、東海道・山陽新幹線：60 Hz・東北・上越新幹線：50 Hzと異なることや、東北・上越新幹線用のものには降雪対策が施された車体設計にしなければならない点などから、1996年に計画の中止が発表された[28]。計画については、「東北新幹線#東海道・山陽新幹線との直通運転」も参照。
新幹線直行特急（ミニ新幹線）

以下の2路線が開業している。過去には東北新幹線の盛岡以北および北陸新幹線の軽井沢以西もミニ新幹線として建設することが検討されたが、前者は地元の積極的な運動[29]、後者は長野オリンピックとの兼ね合いがあり最終的にはフル規格で建設された。
名称 	起点 	終点 	営業キロ 	駅数 	開業年月日 	正式路線名 	運営会社
山形新幹線 	福島駅 	新庄駅 	148.6 km 	11 	1992年7月1日：福島駅 - 山形駅
1999年12月4日：山形駅 - 新庄駅 	奥羽本線（山形線） 	東日本旅客鉄道（JR東日本）
秋田新幹線 	盛岡駅 	秋田駅 	127.3 km 	6 	1997年3月22日 	田沢湖線：盛岡駅 - 大曲駅
奥羽本線：大曲駅 - 秋田駅 	東日本旅客鉄道（JR東日本）
新幹線規格在来線

新幹線の回送線で旅客扱いを行う区間。距離が短く高速運転を行わないなどといった理由で在来線扱いとされているが、車両や設備は新幹線のものであるため、これらの路線を走る列車は一般の「特急列車」扱いとされ、乗車の際には乗車券のほかに特急券を要する。
名称 	起点 	終点 	営業キロ 	駅数 	開業年月日 	運営会社
上越線（支線）[* 1] 	越後湯沢駅 	ガーラ湯沢駅 	1.8 km 	2 	1990年12月20日 	東日本旅客鉄道（JR東日本）
博多南線[* 2] 	博多駅 	博多南駅 	8.5 km 	2 	1990年4月1日 	西日本旅客鉄道（JR西日本）

    ^ 上越線支線は保守用の引き込み線を旅客線化したもの。通称・ガーラ湯沢支線。時刻表には運行上の形態にあわせ「上越新幹線」で掲載されており、「上越線」では掲載されていない。
    ^ 博多南線は車両基地（博多総合車両所）への回送線を旅客線化したもので、路線の大半が九州新幹線鹿児島ルートとの共用。全列車が、JRでは唯一列車愛称がない特急列車。

新幹線では通常、東京駅 - 上野駅間や東京駅 - 品川駅間などの短距離区間であっても、自由席特定特急料金として870円が必要となる。しかし上記の区間は在来線特急扱いであるため、特定特急料金がJRの特急料金では最低の100円となる。
新幹線鉄道規格新線

北陸新幹線と九州新幹線鹿児島ルート・西九州ルートのそれぞれ一部区間は、当初この方式で着工されたが、後に標準軌新線（フル規格）に変更されたため、この方式で開業した新幹線路線は存在しない。

以下の路線は通常の在来線として開業したが、将来の新幹線路線の敷設を考慮し、車両限界等が新幹線規格で建設されている。なお、海峡線の新中小国信号場 - 木古内駅間 (85.5 km) は2016年（平成28年）3月26日から北海道新幹線と共用されている。
名称 	起点 	終点 	営業キロ 	開業年月日 	運営会社
海峡線 	中小国駅[* 1] 	木古内駅 	87.8 km 	1988年3月13日 	北海道旅客鉄道（JR北海道）
本四備讃線 	茶屋町駅 	児島駅 	12.9 km 	1988年3月20日 	西日本旅客鉄道（JR西日本）
児島駅 	宇多津駅 	18.1 km 	1988年4月10日 	四国旅客鉄道（JR四国）

    ^ 中小国駅 - 新中小国信号場 (2.3 km)は東日本旅客鉄道（JR東日本）津軽線との重複区間である。

計画路線

1970年（昭和45年）5月18日に公布された全国新幹線鉄道整備法に基づき基本計画線が挙げられたが、オイルショックや国鉄の経営悪化などの影響を受けて、以下の新幹線の着工は見送られた。このうち整備新幹線は平成に入って着工したが、基本計画のまま着工の目処が全く立っていない路線も存在する。この区間については建設を望む声が根強く残っている区間や、フリーゲージトレインによる新幹線との直通運転が提案されている区間も存在する。
整備計画路線

1973年（昭和48年）11月13日に整備計画が決定したいわゆる「整備新幹線」と、2011年（平成23年）5月26日に整備計画が決定した中央新幹線がある。
整備新幹線 名称 	起点 	終点 	線路延長 	開業予定 	状況 	営業主体
北海道新幹線 	新函館北斗駅 	札幌駅 	211.5 km 	2030年度 	建設中 	北海道旅客鉄道（JR北海道）
北陸新幹線
敦賀駅 	新大阪駅 	131.0 km 	2046年頃 	未着工 	西日本旅客鉄道（JR西日本）
九州新幹線
（西九州ルート） 	新鳥栖駅 	武雄温泉駅 			未着工 	九州旅客鉄道（JR九州）
名称 	起点 	終点 	線路延長 	開業予定 	状況 	営業主体
中央新幹線[* 1] 	品川駅 	名古屋駅 	285.6 km 	2027年以降 	建設中 	東海旅客鉄道（JR東海）
名古屋駅 	新大阪駅 	約152 km 	2037年〜2045年 	未着工

    ^ 設計最高速度505 km/hの超電導磁気浮上式鉄道（リニアモーターカー）。一部は山梨リニア実験線を転用予定。

基本計画路線
名称 	起点 	終点 	線路延長
北海道新幹線 	北海道札幌市 	北海道旭川市 	約130 km
北海道南回り新幹線 	北海道山越郡長万部町 	北海道札幌市 	約180 km
羽越新幹線 	富山県富山市 	青森県青森市 	約560 km
奥羽新幹線 	福島県福島市 	秋田県秋田市 	約270 km
北陸・中京新幹線 	福井県敦賀市 	愛知県名古屋市 	約50 km
山陰新幹線 	大阪府大阪市 	山口県下関市 	約550 km
中国横断新幹線 	岡山県岡山市 	島根県松江市 	約150 km
四国新幹線 	大阪府大阪市 	大分県大分市 	約480 km
四国横断新幹線 	岡山県岡山市 	高知県高知市 	約150 km
東九州新幹線 	福岡県福岡市 	鹿児島県鹿児島市 	約390 km
九州横断新幹線 	大分県大分市 	熊本県熊本市 	約120 km
未成線
	
この節は検証可能な参考文献や出典が全く示されていないか、不十分です。出典を追加して記事の信頼性向上にご協力ください。（このテンプレートの使い方）
出典検索?: "新幹線" – ニュース · 書籍 · スカラー · CiNii · J-STAGE · NDL · dlib.jp · ジャパンサーチ · TWL（2017年1月）

成田新幹線（東京駅 - 成田空港駅間）
    1971年（昭和46年）1月18日、昭和46年告示第17号により基本計画が公示された。1974年（昭和49年）に着工したが、オイルショックの影響や、用地取得の困難、沿線自治体の建設反対運動が激しかったこともあり、1983年（昭和58年）に工事は中止され、その後1987年（昭和62年）の国鉄分割民営化に伴い、基本計画が失効した。建設済みの施設は成田空港高速鉄道線（成田線空港支線）に転用され、新幹線の東京駅が建設される予定であったスペースには京葉線東京駅が後に建設された。なお、元神奈川県知事の松沢成文や、公約に「羽田・成田リニア新線構想」を掲げて当選した千葉県知事の森田健作がリニア検討委員会の発足を検討している[30]。
第二東海道新幹線
    リニアモーターカーで建設される計画であったが、前述した中央新幹線の計画（山梨リニア実験線の活用）と統合された。
上越新幹線（新宿駅 - 大宮駅間）
    上越新幹線は線路容量とターミナル容量から、当面新宿駅 - 大宮駅間の工事実施計画申請は行わない[31]こととし、東北新幹線に乗り入れとなったが、新宿駅地下にもスペースが確保されていた[注 5]。1973年（昭和48年）7月12日、参議院運輸委員会において内田隆滋国鉄理事は、「大宮と東京の間は新幹線が二本要る、それを施行する場合、地元の御要望もございまして、いわゆる現在の計画している新幹線と通勤線とを併設いたしまして、現在の貨物線にもう一本の新幹線を通すという計画をいたしたわけでございます。」と答弁している。また1973年（昭和48年）9月4日、参議院運輸委員会において磯崎叡国鉄総裁は「東北、上越あるいは北陸を東京都内に入れる際に、東京都から、東京駅に集中しないでほしいという強い御要請がございました。その際に、私どももそれを受けまして、東北、上越の認可を大臣からいただきます際には、将来そのターミナルを東京駅以外のところに持っていきたいということをつけ加えて大臣の御承諾を得ております。その際に、やはりその第一候補となるのは新宿でございます。」と答弁している。なお、1971年（昭和46年）9月15日付け読売新聞では、「国鉄、鉄建公団では、51年の開通当初は東北、上越両線を東京駅から大宮駅付近まで併用とし、新宿駅完成後に東北新幹線は東京駅、上越新幹線は新宿駅に分離したい考え。東京駅を出た列車は、高架から秋葉原付近でいったん地下にもぐり、田端付近でカオを出すが、分離後の上越新幹線は、この田端を分岐点に、山手貨物線あとをたどり新宿駅と結ばれる。さらに新宿から山手貨物線あとを延長し大崎付近で東海道新幹線と直通させる計画も出ている。 」とし、1973年（昭和48年）3月11日付け毎日新聞においては、「来年10月の武蔵野線全面開通に伴い不要化する東北本線の貨物線と山手貨物線の敷地を使い、大宮-赤羽（以上東北本線）赤羽-池袋-新宿（以上山手貨物線）というコースをとる」とされている。整備新幹線開業後の大宮 - 東京間および東京駅の容量逼迫に備えてこの区間の建設を再開すべきだという意見がある。ただし、埼京線高架沿いの空き地は「都市施設用地」と呼ばれる都市施設（道路、公園等）を計画したものであり、延伸のために確保された用地ではなく、国鉄が先行取得し、いずれ、戸田、浦和、与野、大宮の4市（浦和市、与野市、大宮市は現・さいたま市）に売却することとされている[32][33]。また、前記の「不要化する貨物線」とされている線路については現在東北・上越新幹線と並行する東北本線（宇都宮線）・高崎線の一部列車（湘南新宿ライン）が、更に池袋駅以南は埼京線も乗り入れ、重要な通勤路線として再活用されており、上越新幹線への転用は困難となっている。他方、2017年（平成29年）1月12日付け信濃毎日新聞では「JR東日本は、巨額の投資が必要な大宮以南の線路増強について慎重だ。「現有の新幹線設備を有効活用することで、利用状況を踏まえた列車本数を確保できると考えている」（広報部）とする。今後、需要が増える場合には「大宮発着の列車の運行などを必要に応じて検討していくことになる」との立場だ。」としている。鉄道総合技術研究所の小野田滋は「このルートは工事費が巨額となることから現実的ではないと判断され打ち切られ」[34]たとしている。

列車名

新幹線の列車名（列車愛称）は、東海道・山陽・九州新幹線では速度別につけられている。JR東日本の山形・秋田新幹線およびJR九州の西九州新幹線は列車名が単一である。JR東日本の北陸新幹線とJR九州の九州新幹線も部分開業時は列車名が単一であった。JR東日本の新幹線では、E1系・E4系「Max」を使用する場合は列車名の前に「Max」が付いていた。
かつて存在した"グランドひかり"の食堂車
東海道新幹線最終日の初代新幹線0系「こだま」
現行の列車名

東海道・山陽新幹線

    「のぞみ」
        現行の東海道・山陽新幹線の最速列車で、「ひかり」「こだま」とは異なる料金体系である。1992年3月14日に運行開始された。N700系、N700S系が使用される。過去には300系、500系、700系も使用されていた。
    「ひかり」
        主要駅停車の列車。各駅停車ではないが、「のぞみ」に比べ停車駅が多く、料金体系も異なる。東海道新幹線開業当初から運行されており、当初は大都市駅のみに停車し、各駅停車の「こだま」に対して超特急の代名詞であった。その後、運行本数の増加に伴い、主要駅のみ停車する速達タイプ、乗降客数の少ない駅にも停車するタイプ、一部の区間で各駅に停車するタイプなど停車駅が多様化し、「のぞみ」が加わった以降では「のぞみ」でも「こだま」（各駅停車）でもない列車という位置づけになっている。山陽新幹線では"ひかりレールスター"と呼ばれる、顧客ニーズに応える形で登場した列車も運行されている。過去には"ウエストひかり"や"グランドひかり"などもあった。「のぞみ」と違い、一部区間が各駅停車となる列車もある。N700系・N700S系・700系（山陽新幹線のみ）が使用される。過去には0系、100系、300系、500系も使用されていた。
    「こだま」
        各駅停車の列車で、「ひかり」と同様、東海道新幹線開業当初から運行されている。全区間にわたって運行されているが、東海道区間と山陽区間（新大阪駅）を跨ぐ列車はない。早朝や深夜には普通車が全車自由席の列車や、グリーン車なしで全車自由席の列車も運行される。N700系、N700S系が使われるほか、山陽新幹線では500系（8両編成）、700系、N700系7000番台・8000番台を使用する列車がある。2008年11月30日までは0系、2012年3月までは100系・300系も使用されていた。

山陽・九州新幹線

    「みずほ」
        新大阪駅 - 鹿児島中央駅を結ぶ最速列車。九州新幹線内では久留米駅（一部列車のみ）・熊本駅・川内駅（一部列車のみ）・鹿児島中央駅に停車する。朝夕を中心に運転される。新大阪駅 - 鹿児島中央駅間を最速3時間41分で結ぶ。山陽新幹線内では「のぞみ」並みの所要時間で、料金体系も「のぞみ」と同一である。N700系7000番台・8000番台を使用。
    「さくら」
        主要駅停車の列車。山陽・九州新幹線の直通列車と、九州新幹線内のみ運行する列車がある。山陽新幹線内では「ひかり」並みの所要時間で、料金体系も「ひかり」と同一である。九州新幹線内では熊本駅 - 鹿児島中央駅間（熊本駅発着の1往復は博多駅 - 熊本駅間）が各駅停車となる列車もある。N700系7000番台・8000番台と800系を使用する。なお、800系は九州新幹線内のみを運転する一部列車で使用される。
    「つばめ」
        各駅停車の列車。ほとんどが九州新幹線内のみの運行で、早朝・深夜を除いて博多駅 - 熊本駅間の運行であるが、朝に山陽新幹線直通の熊本発小倉行きが1本運行されている。主に800系を使用するが、一部列車はN700系7000番台・8000番台も使用する。

西九州新幹線

    「かもめ」
        運行区間・停車パターンに関係なく全ての列車に「かもめ」の愛称が使用される。武雄温泉駅で「リレーかもめ」[注 6]と同一ホーム対面乗り換えを行っている。N700S系8000番台を使用する。

東北・北海道新幹線

    「はやぶさ」
        東京駅 - 仙台駅・盛岡駅・新青森駅・新函館北斗駅間を結ぶ最速列車。2011年3月5日に運行開始した。E5系、H5系を使用する。東北新幹線の他の列車とは異なる料金体系をとる。早朝・夜間の一部を除き全車指定席で、グリーン車のほかに、グランクラスがある。ただし一部の「はやぶさ」のグランクラスはシートのみの営業となる。
    「はやて」
        盛岡駅・新青森駅 - 新函館北斗駅間を運行するが、臨時列車としては東京駅 - 新青森駅間で大宮駅 - 仙台駅間ノンストップの列車が運行される場合もある。2002年の八戸開業時に東京駅 - 八戸駅間を結ぶ列車として運行開始された。前述の「はやぶさ」の登場および運行本数拡大によって少数派となったが、2014年11月20日の発表では、北海道新幹線開業以降も存続することとなっている[35][36]。早朝・夜間の一部を除き全車指定席。E5系、E2系が使用されている。E5系を使用する列車にはグランクラスが連結されているが、一部列車のグランクラスは非営業となる。
    「やまびこ」
        盛岡駅以南を走る列車で、下記の「なすの」を除くもの。仙台駅以南では主要駅停車、以北では各駅停車が多い。全区間各駅停車の列車もある。東北新幹線開業当初から運行されており、当初は各駅停車の「あおば」に対して主要駅停車もしくは一部区間で各駅停車の列車という位置づけであった。E5系、E6系、E2系を使用。E5系を使用する列車にはグランクラスが連結されているが、一部列車のグランクラスはシートのみの営業または非営業となる。
    「なすの」
        東京駅 - 那須塩原駅・郡山駅間を走る各駅停車の列車。E5系、E6系、E2系が使用される。2010年までは増結用として400系も使用された。E5系を使用する列車のグランクラスはシートのみの営業となる。

山形新幹線

    「つばさ」
        山形新幹線開業時から運行されており、停車駅や使用車両に関係なく、山形新幹線内を通るすべての新幹線列車にこの愛称が付けられている。E3系1000番台および2000番台を使用し、福島以南は基本的にE2系「やまびこ」と併結（一部は単独運転する）。2010年までは400系も使用され、過去には200系「やまびこ」やE4系「Maxやまびこ」と併結していた。

秋田新幹線

    「こまち」
        秋田新幹線開業時から運行されており、停車駅や使用車両に関係なく、秋田新幹線内を通るすべての新幹線列車にこの愛称が付けられている。E6系を使用し、盛岡以南は基本的にE5系「はやぶさ」と併結。2014年3月まではE3系も使用され、過去には200系「やまびこ」やE2系「やまびこ」→「はやて」、E5系「はやて」と併結していた。全車指定席。

上越新幹線

    「とき」
        越後湯沢駅以北を運行する列車。ほとんどが東京駅 - 新潟駅間で運行されている。主要駅停車の列車と各駅停車の列車がある。E2系、E7系を使用。過去には200系、E1系、E4系も使用されていた。上越新幹線開業時から運行されており、開業当初は、主要駅停車の「あさひ」に対し、各駅停車の列車名として使用されていた。1997年10月1日に「たにがわ」「あさひ」への統合で消滅したが、東京駅から高崎駅まで同じ区間を運行する「あさま」との混同を防止するため、2002年12月1日のダイヤ改正で「あさひ」からの改称という形で復活した。
    「たにがわ」
        越後湯沢駅（スキーシーズンはガーラ湯沢駅）以南を走る列車。使用車両は「とき」と同じ。定期列車は一部列車以外は各駅停車で[注 7]、普通車全車自由席の列車もある。冬季には一部の途中駅を通過するガーラ湯沢駅発着の臨時列車も設定されている。

北陸新幹線

    「かがやき」
        東京駅 - 金沢駅・敦賀駅間直通列車のうち、主要駅に停車する速達タイプ。E7系、W7系を使用。グランクラスを設定。全車指定席。
    「はくたか」
        東京駅 - 金沢駅・敦賀駅間直通列車のうち、停車タイプ。北陸新幹線内のみ運行の金沢発長野行きが夜に1本運行されている。使用車両は「かがやき」と同じE7系、W7系を使用。グランクラスも設定されている。
    「あさま」
        東京駅 - 長野駅を結ぶ列車。E7系、W7系を使用し、グランクラスはシートのみの営業。過去にはE2系、臨時列車で200系、E4系が使用されたこともあり、E4系使用の場合「Maxあさま」の名前が付いた。
    「つるぎ」
        富山駅 - 敦賀駅間で運転されるシャトルタイプの列車。使用車両は「かがやき」「はくたか」と同じ。グランクラスはシートのみの営業。

かつて使用されていた列車名

東北新幹線

    「あおば」
        各駅停車の列車。1997年10月1日のダイヤ改正で「なすの」・「やまびこ」へ統合して消滅した。

上越新幹線

    「あさひ」
        速達型列車として設定されたが、長野新幹線開業後は越後湯沢駅以北の区間を走行する列車の統一名称となった。「あさま」と名称が紛らわしいため、2002年12月1日のダイヤ改正で「とき」と改称された。ただし、前述の「たにがわ」に該当する列車は、それ以前から改称されていた。

秋田新幹線

    「スーパーこまち」
        E3系0番台からE6系への置き換えの過渡期に設定された愛称。E6系で東北新幹線を300 km/hで運行する列車を区別するためのもので、E6系への置き換えが完了した2014年3月15日のダイヤ改正で全て「こまち」に戻された。

歴史
経緯・背景
戦前における高速鉄道

日本の鉄道は明治時代の草創期にコストの面から狭軌を採用したため、その規格の低さに加えて地形的な事情から勾配や曲線が多いなどの制約を受け、欧米の鉄道のような高速運転とは無縁であった。最高速度は1910年代から1950年代まで100 km/h以下に留まっていた。

そこで標準軌に改軌する提案も、明治から大正にかけて何度か出されていたが、政争や予算問題などから結局実現しなかった（日本の改軌論争も参照）。

また1910年代には、東京 - 大阪間に電車による高速新路線「日本電気鉄道」を敷設する計画が民間から出されたが、国の許可するところとならず、実現には至っていない。

日本における現実的な高速列車開発は、日本の勢力下にあった満洲（現在の中国東北部）を縦断する南満洲鉄道（満鉄）に始まる。同社は日本の資本と技術により運営されており、ほとんどの幹部・技術者が日本人で、実質的に日本の鉄道と言っても過言ではない。

当時の満鉄は電化以前の鉄道で蒸気機関車牽引であったが、1,435 mmの国際標準軌（日本では広軌と称した）を用いた高規格路線であり、保守的な日本内地の鉄道省とは一線を画した先進的な試みを早くから行っていた。

1934年（昭和9年/康徳元年）、満鉄は自社設計によって当時の欧米の潮流に互した流線形蒸気機関車「パシナ形」を開発、これに新開発の流線形客車編成を組み合わせ、大連 - 新京（現・長春）間701 kmに特急「あじあ」号を運転開始した。この列車は最高速度120 km/h以上を誇り、最高95 km/hに留まる鉄道省の列車をはるかに凌駕した。所要8時間30分、表定速度は82 km/hに達した。

とはいえ、当時の欧米の鉄道はさらに上を行っていた。例えばイギリスのロンドン・アンド・ノース・イースタン鉄道（LNER）がロンドン - エディンバラ間に運転していた特急列車「フライング・スコッツマン」は、蒸気機関車牽引で最高速度160 km/h以上での営業運転を行っており、ドイツ国鉄では気動車列車「フリーゲンダー・ハンブルガー」が150 km/h 以上の高速で営業運転していた。さらにアメリカの私鉄各社には、定期運転列車を牽引して、最高速度は優に180 km/hに達する蒸気機関車さえ存在していたのである。

「あじあ」号は全客車冷暖房完備など世界の最先端を行っていた部分もあったが、120 km/h運転そのものは、当時の欧米の主要幹線での標準的な水準に達したものでしかなかった。

この技術が、日本本土の鉄道に直接生かされることはなかった。しかし満鉄関係者には鉄道技術者の島安次郎がおり、その長男の島秀雄と共に後述する「弾丸列車計画」を推し進めることになる。

なお、前述した日本電気鉄道のように、民間による大規模な都市間電車は実現しなかったが、中近距離の都市間電車に関しては、新京阪鉄道[注 8]や阪神急行電鉄、参宮急行電鉄、阪和電気鉄道のように、アメリカのインターアーバンの技術を取り入れるなどして実現させた所もあった。これら路線の多くは、既存の鉄道線と競合する形で敷設されたものとなっており、「（既存の並行線よりも）高規格な路線において、高速運転を行うこと」がその建設目的となっていた。「新しい高規格線を敷く」という意味では、新幹線に通じる所もある。

その中でも、参宮急行電鉄が転じた関西急行鉄道は途中に伊勢中川駅での乗り換えこそあるものの、大阪と名古屋という中距離の2大都市間（当時の営業キロで189.5 km）を電車で結ぶことに成功しており、また阪和電気鉄道は「あじあ」号の水準に匹敵する、表定速度81.6 km/hの「超特急」を狭軌路線で運転していた。

これらの私鉄で用いられた電車はハイレベルな仕様の車両が多く（新京阪P-6形、参急2200系、阪和モヨ100形など）、後述する国鉄における動力分散方式の開発にも、いくらか影響を与えている。
弾丸列車計画
詳細は「弾丸列車」を参照

1930年代に入ると、満洲事変や日中戦争の激化などにより、日本から中国や満洲国へ向かう各種物資輸送需要の激増で、東海道・山陽本線の輸送量も増大した。

このころ鉄道省内部に「鉄道幹線調査会」が設立され、主要幹線の輸送力強化についての検討が行われた。ここから抜本的な輸送力増強手段として1939年に発案されたのが「弾丸列車計画」であった。

これは、東京から下関まで在来の東海道・山陽本線とは別に広軌（1,435 mm・標準軌）の新路線を建設し、最高速度200 km/hと満鉄「あじあ」号を超える高速運転を行い、東京 - 大阪間を4時間、東京 - 下関間を9時間で結ぶことを計画したものであった。この計画は翌1940年（昭和15年）9月に承認され、建設工事が始められることになった。

すでにこの時点で、新しい幹線を敷設するということから「新幹線」や「広軌新線」という呼称を内部関係者は用いていた。「新幹線」の語はここが起源であるとされている。

また将来的には対馬海峡に海底トンネルを建設して、統治下である朝鮮半島へ直通、釜山から奉天（現：瀋陽）を通り満洲国の首都新京（現：長春）、さらには北京・昭南（現：シンガポール）に至る、という構想も一部では描かれていた。

当時の鉄道では、日本においてもまだ機関車が客車を牽引する方式が一般的であったうえ、完全電化したものの、発電所が敵国からの攻撃を受けた場合の対処について軍からクレームを受けたために、「弾丸列車」も電気機関車と蒸気機関車を併用する方式で計画された。

1941年12月の太平洋戦争勃発後も工事は続けられ、日本坂トンネルや新丹那トンネルが、1942年には東山トンネルの工事が着工したが、最終的には戦況の悪化で中断した（その後再開され新幹線に利用）。しかし、そのルートの相当部分が後の東海道新幹線建設で役立てられた。特に、土地買収が戦時中の時点で半ば強制的な形で相当な区間において終わっていたことは、新幹線建設をスムーズにした。

この弾丸列車計画の技師たちが居住した地として、静岡県田方郡函南町には「新幹線」という地名が、東海道新幹線の開業前から存在した。
動力分散化への流れ

太平洋戦争終結後数年間、鉄道をも含めて混乱の極みにあった日本も、1950年（昭和25年）の朝鮮戦争以降本格的に復興し、鉄道の都市間輸送需要も急激に伸張していった。

旧日本軍の研究部門や軍需企業、旧南満洲鉄道に所属し、戦後その職を失ったり技術を持て余していた優秀な人材を、昭和20年代の国鉄が多数獲得したことは見逃せない事実である。高速走行中の車両の振動や、空力特性の研究は、旧軍出身技術者の存在によって大きく進展した。

1955年（昭和30年）に国鉄総裁に就任した十河信二は、国鉄出身の卓越した技術者であるが一時民間にあった島秀雄を再度招聘し、国鉄技師長に就任させた。彼らを中心とする人々が、その後新幹線計画を推進することになる。

地盤が悪く山がちな日本において列車を高速運転するには、機関車が客車を牽引する「動力集中方式」よりも、電車・気動車のように編成の各車両に動力を持たせる「動力分散方式」の方が適している。カーブや勾配の多い条件でも加減速能力に優れ、また線路への負担が小さいため、脆弱な地盤に敷かれた線路でも高速を出せるからである。当時は蒸気機関車主流の時代であり、また国際的に見ても主流であることから、国鉄部内でも動力集中式に固執する者が多かったが、島秀雄は例外的に戦前から動力分散方式の特性を理解し、研究していた。

島は1951年（昭和26年）に事情によって国鉄を離れていたが、彼の指揮の下で1950年（昭和25年）に開発された東海道線普通列車用の80系電車は、短距離向けと見られていた電車が、長距離運転にも優れた特性を発揮するという事実を実証し、その後国鉄の在来線に電車・気動車の普及を進める原動力となった。島の復帰以降、国鉄の動力分散化の流れはさらに加速する。
1950年代の高性能電車の出現

日本では1953年（昭和28年）以降、欧米からの新技術移入や国内メーカーの技術開発に伴い、電車の高性能化の動きが始まった。

この過程で、振動を抑制し、乗り心地改善と高速運転に資する「カルダン駆動方式」と高速対応の新型台車、床面シャーシだけでなく側板や天井にも応力を分散させた「全金属製軽量車体」、全車両にモーターを搭載して加速力を高める「全電動車方式」、反応速度が速い上に取り扱いが容易な「電磁直通ブレーキ機構」、制御装置1台を2両の電動車で共用して軽量化やコストダウンを実現する「1C8M方式（MM'ユニット方式）」など、それ以前の電車とは一線を画する重要な革新的技術が、1953年（昭和28年）からわずか数年の間に実用化された。

この結果、高速性能・加減速性能に優れ、しかも居住性の良い高性能電車が、1954年（昭和29年）以降大手私鉄を中心に続々と出現して、大きな技術的成功を収めた。国鉄もこの潮流に乗って高性能電車の開発に取り組み、1957年（昭和32年）に新型通勤電車モハ90系（後の101系）を完成させる。
小田急ロマンスカー3000形SE

同年に小田急電鉄が完成させた低重心・連接構造の流線型特急電車3000形「SE車」は、鉄道技術研究所の技術指導を受けて設計された[38]車両で、最高速度145 km/hを目指した野心作であった。しかし、曲線の多い小田急の路線ではその高速性能は十分に発揮できなかった[39]ため、小田急から国鉄に対して、試験で収集されたデータを小田急と国鉄の双方で利用することを条件として[39]、軌道条件の優れている国鉄の路線上での走行試験について申し入れがあった[40]。

国鉄はこの申し入れに対して快諾[41]、小田急からSE車を借り入れ、1957年（昭和32年）9月に東海道本線で速度試験を行った。結果SE車は計画通りの145 km/hに到達し[42]、当時の狭軌鉄道における世界速度記録を達成した。続いて国鉄はモハ90系通勤電車をギア比変更などで高速化改造、空気抵抗の面で不利な形態ながら135 km/hの好記録を達成した。

これらの実績を踏まえて、1958年（昭和33年）にはモハ90系の技術を応用し、東海道本線特急「こだま号」用に国鉄初の特急形電車モハ20系（後の151系）が開発された。流線型の軽量・低重心な車体は冷暖房完備で、空気ばね台車も装備し、スピードと快適な乗り心地を両立させて、動力集中方式の客車列車を完全に凌駕した。翌1959年7月には、東海道本線での速度試験で通常編成（4M4Tの8両編成）からデッドウェイトとなる付随車2両を抜いた6両編成で最高速度163 km/hに達し、小田急SE車の速度記録を更新した。

これらの電車における顕著な成績は、動力分散方式の資質を実証し、ひいては新幹線車両に電車を用いることへの強力な裏付けとなった。

1955年（昭和30年）から国鉄は交流電化方式の実用化に独自に取り組む。同年国鉄代表団はフランスパリで、狭軌用交流機関車を買いつける算段をするが、日本側は技術研究のための5両程度の買い付け意欲しかみせず、交渉は決裂[43]。日本では同年中に独自開発による商用周波数の単相交流（20 kV 50 Hz）を使用する電気機関車2両（ED44 1（日立製作所製）・ED45 1（三菱電機・新三菱重工業製））の試作に成功する。

これらの試作車による研究成果を受けて、国鉄は1957年の北陸本線を皮切りに、地方線区での交流電化を開始した。これ自体は従来の直流電化に比べ、変電所間隔を長くできることから地上設備コストが低いと考えられたことによるものであったが、後に新幹線の電化システムに応用されることになる。超高速の電気鉄道においては大量の電力消費が生じ、これに伴って架線から効率よく集電するには、従来から用いられて来た1,500 Vの直流電源より、大電力を長距離送電できる高圧交流電源を用いる方が適していたのである（日本の鉄道の交流電化方式は在来線20 kV、新幹線25 kVで、電圧だけでも直流電化路線の10倍以上のレベルである）。
幹線調査会と鉄道技術研究所の活動と建設計画承認

戦後の復興と共に鉄道および道路輸送の需要が増大すると、当時の日本における最重要幹線であった東海道本線の貨客輸送能力は、ほぼ限界に達していた。1956年に東海道本線の全線電化が完成するが、需要の増加には焼け石に水であった。

1957年（昭和32年）、国鉄内部の「幹線調査会」は、東海道本線の輸送力飽和は早晩必至とし、現在線以外の線路増設が必要であると答申した。実際の手法としてさまざまな案が出されたが、基本的に以下の3案のいずれかが選択されることになった。

    現在線に沿って線路を増設、複々線とする。
    別ルートで狭軌新線を建設する。
    別ルートで広軌新線を建設する。

東海道の線増計画は、従来の常道であれば複々線案が採られたところであった。しかし、十河ら国鉄幹部は将来の発展性を視野に入れ、あえて困難の多い広軌新線建設を決定したのである。それは戦前の弾丸列車計画を、戦後の技術革新の下で、改めて実現しようとする超高速列車計画であった。

同年5月30日には鉄道技術研究所（現：鉄道総合技術研究所）の篠原武司所長らが、鉄道技術研究所創立50周年記念講演「東京 - 大阪間3時間への可能性」で、広軌新線ならば東京 - 大阪間の3時間運転は技術的に可能であるという報告を行った。十河はその話を聞くや強い関心を示し、国鉄幹部を集めて技術研究所員に詳細を話させたという。

当時欧米では、将来の大量輸送手段として航空機と高速道路網による高速輸送が有望視され、鉄道はそれらに取って代わられる時代遅れのものだという見解が広まっていた。日本でもこれを範としようとする向きが一般的であり、在来線とは別規格の高速新線を建設するという計画は、国鉄内部でさえも疑問視する者が多かった。

鉄道ファンでもある作家の阿川弘之ですら、戦艦大和（大和型戦艦）・万里の長城・ピラミッドが「世界三大馬鹿」であり、この時期に莫大な投資をして新幹線を造れば「第2の戦艦大和」となって世界の物笑いの種になると批判した[44]（後に阿川は新幹線が世界の鉄道斜陽論を覆すに至るまでの成功を収めたのを見て、十河の後を継いで国鉄総裁を務めた石田禮助との対談において、自らの不明を悔やむ発言をしている）。

そのような厳しい状況下で、十河と島は東海道に新たな大規模高速輸送用の鉄道路線（新幹線）を実現すべく政治的活動（十河が担当）と、技術的プロジェクト（島らが担当）を続けた。

技術的裏付けの下、1958年（昭和33年）に建設計画が承認され、翌1959年（昭和34年）4月20日に起工式が行われた。総工費は当初予定から修正され、3800億円にまで膨らんだ。元々十河などが国会内での承認を得るために安く見積もっていたこと、地価高騰のあおり、さらには新幹線建設に集中するために地方路線建設の陳情を蹴り国会議員の不興を買っていたこともあって、後には国会で責任問題に発展した。新幹線開業前に責任を取る形で十河は国鉄総裁を退任し、島も十河に殉じて国鉄を退職する。

1961年（昭和36年）5月1日に国鉄はこのプロジェクトに対し、世界銀行から8000万ドル（当時は1ドル=360円の固定相場制）の融資を受けた（この融資は1981年〈昭和56年〉に返済が完了した）。この融資を受けたことで、新幹線プロジェクトは日本の国家的プロジェクトとなり、国内事情によって中断することは許されなくなった。

その建設に関しては前述の通り、戦前の「弾丸列車計画」の際に開削されたトンネルや、買収された用地の多くが活用された。5年という短期間で完成したのは、この時の用地買収および工事があったからだともいわれている。また大阪府・京都府内では、完成した新幹線の線路を高架工事中の仮線として用いて、暫定的に阪急京都本線の電車を走らせていたこともあった（→新幹線の線路を先に走った阪急電車）。
モデル線鴨宮基地整備
鴨宮モデル線

1962年（昭和37年）には神奈川県綾瀬付近 - 小田原付近の区間がモデル線として先行整備され、小田原市鴨宮（かものみや）に鴨宮基地とそれを統括するモデル線管理区が置かれた。小田原 - 綾瀬間が試験路線に選ばれた理由は以下の通りである。

    戦前の弾丸列車構想に際してすでに用地を取得しており、早い時期に着工する事が可能である。
    直線・カーブ・トンネル・鉄橋と、線形や地上設備のシチュエーションが一通り揃っており、データ収集が容易である。
    鴨宮付近では東海道本線と隣接しており、車両・資材などの搬入に便利である。
    国立にある鉄道技術研究所からも近く、問題が発生した時も対処が容易である。

ここで2編成の試作電車「1000形」を用いて車両と設備のテストを繰り返し、問題点をあぶり出しては改良を重ねていった[45][46]。1963年（昭和38年）3月30日の速度向上試験では、1000形B編成が256 km/hの国内速度記録を達成している。モデル線での研究は、初代量産形新幹線電車となる0系や、線路設備の開発に生かされることになった。また、ここに中央鉄道学園小田原分所を設けて、新幹線のための乗務員と保線要員の養成も同時に行った。

しかし、このモデル線には欠点があった。相模湾に近く、冬でも比較的温暖な鴨宮では、降雪時の高速運転を想定した試験データは十分に得られなかったのである。東海道新幹線の名古屋 - 新大阪間経路は、当初計画した鈴鹿山脈経由ルートが費用や技術・工期の制約から断念され、東海道本線同様に関ヶ原を経由するルートに変更されていた。関ヶ原周辺は谷間で標高も高く、さらに日本海側気候の影響で冬期には激しい降雪のある地域でもある。このような区間を冬期に高速列車で通過する状況の研究が、開業前には十分に行えなかった。このことは、1964年（昭和39年）の開業後初めての冬期に関ヶ原での着雪による車両故障を頻発させる原因となった。

このモデル線区は、設備が無駄にならないよう、建設中の路線の一部を先行完成させて利用する手法が採られ、東海道新幹線開業後は新横浜 - 小田原間の一部に組み込まれている。この手法は後続の東北新幹線の小山実験線や、リニア山梨実験線にも踏襲されている。小山実験線には実際に駅施設も設けられ、後に小山駅となった。現在、鴨宮基地のあった場所は保線車輌の基地となっており、その一角には新幹線モデル線を示すモニュメントが設置されている。2009年（平成21年）5月には市民の手によって、0系新幹線の前頭部をモチーフとした「新幹線発祥の地」のモニュメントが建てられた。

またテストに使われた試作電車は、東海道新幹線開業後に改造を受けた。A編成は救援車941形に、B編成は電気軌道総合試験車922-0形となり、それぞれ役立てられることになる。941形はまったく活躍することなく廃車となったが、922-0はその後0系を元とした「ドクターイエロー」が登場するまで生き永らえた。
新幹線開業
旅客用時刻表に付属した、1964年10月1日より有効の英語版国鉄路線図。東海道新幹線が赤色の線で示されている
1964年10月1日に開業した東海道新幹線が示されている英語の国鉄旅客時刻表
青とクリーム色のカラーリングの初代0系

日本国有鉄道（国鉄）時代の1964年（昭和39年）10月1日に開業した東海道新幹線が初の路線である[47][48]。開業に先立ち、東京駅の東海道新幹線８、９番ホームの間に「０キロメートル」の起点標（距離標）が設置された。開業を1964年にしたのは、1964年の東京オリンピックの開催時期に合わせたからである。

併せて専用の0系が開発され、営業に投入された（→1964年10月1日国鉄ダイヤ改正も参照）。なお、開通に先立つ同年4月22日からアメリカのニューヨーク市で開催されたニューヨーク万国博覧会の日本館に実物大モックアップが展示され、日本の技術力を誇示した。

10月1日の東京発の一番列車（ひかり1号、運転士・山本幸一、伊月正司の2名）は定員987名のところ乗客は730名ほどであり満席ではなかった。

開業当初の営業最高速度は200km/h（東京 - 新大阪間「ひかり」4時間[49][50]、「こだま」5時間[50]）。路盤の安定を待って翌年に210km/h運転（同「ひかり」3時間10分[51]、「こだま」4時間[51]）を開始した（→1965年10月1日・11月1日国鉄ダイヤ改正も参照）。

日本の二大都市である東京 - 大阪間は、1958年（昭和33年）から在来線の特急で日帰り可能になっていたものの、滞在時間がわずか2時間あまりしか取れなかった。しかし新幹線の開通により、日帰りでも滞在時間を充分取れるようになり、社会構造に著しい変化を及ぼした。ビジネスやレジャーの新しい需要を喚起し、東海道新幹線においては当初の12両編成が、1970年（昭和45年）の大阪万博の開幕を機に16両編成まで拡大され、高速大量輸送機関としての確固たる地位を確立した。

その一方で、新幹線の建設や特急・急行列車の増発、さらには都市部における通勤輸送増強（通勤五方面作戦など）などの設備投資に追われたことから、新幹線の開業した1964年（昭和39年）度から国鉄収支は赤字に転落し、以後それは拡大する一方となり、結果的に新幹線建設は国鉄破綻の1つの原因となったと言われる。これに対し、JR東海の葛西敬之会長は著書の中で「東海道新幹線はあくまで内部留保された資金と借金で建設資金をまかない、それらを運賃・料金収入のみですべて回収したものであり、新幹線建設が国鉄破綻の引き金を引いたという認識は誤りだ」と指摘している。いずれにせよ、以後の国鉄において、新幹線は重要な収入源ともなっていく。

その後、東海道新幹線に続いて、同じように需要の増加していた山陽本線の抜本的輸送力改善と高速化を目的として、1967年（昭和42年）に東海道新幹線を延伸する形で山陽新幹線が着工され、1972年（昭和47年）3月15日に岡山まで、1975年（昭和50年）3月10日には博多まで開業した（→1972年3月15日国鉄ダイヤ改正・1975年3月10日国鉄ダイヤ改正も参照）。「ひかりは西へ」がそのキャッチコピーであった。

さらに東北方面への延伸も計画された。1971年（昭和46年）には東北新幹線と上越新幹線が着工され、キャッチコピーには「ひかりは北へ」が使用された（実際の開業にあたっては、「やまびこ」「あおば」「とき」「あさひ」等、東北・上越在来線特急のかつての名称を継承している）。1974年（昭和49年）には建設中の成田空港へのアクセス路線として成田新幹線も工事に入った。折しも田中角栄内閣総理大臣によって、国土開発を促進する「日本列島改造論」が提唱され、整備は順調に進むかに見えた。

だが、実際には反対運動による用地買収の難航やトンネル工事での異常出水などがあり、前者2つの新幹線は予定より工事が5年も遅れ、成田新幹線に至っては工事中止となってしまった（ただし、後にJR東日本と京成電鉄の成田空港乗り入れの際にこの新幹線建設で作られた設備が生かされることになる）。また、名古屋新幹線訴訟など、新幹線沿線での騒音・振動による公害問題がこの頃深刻化した。さらに国鉄財政の悪化に伴う運賃・料金値上げの繰り返し、労働紛争によるストライキの頻発化などから、既存新幹線の乗客が減少傾向に陥った。そして経営問題と労働紛争の影響から技術革新も見られなくなり、新幹線の発展・発達は一時停滞した。

東北新幹線と上越新幹線は1982年（昭和57年）に大宮発着という暫定的な形で開業し（→1982年11月15日国鉄ダイヤ改正・新幹線リレー号も参照）、1985年（昭和60年）には用地買収の関係で遅れていた都心（上野）乗り入れを果たした（→1985年3月14日国鉄ダイヤ改正も参照）。これにより東北・上越地方における鉄道シェアは大幅に拡大した。だが、それら新幹線の建設費負担も重なって、国鉄財政はついに破局的状態となり、中曽根内閣の下で断行された1987年（昭和62年）の国鉄分割民営化に至るのである。
JR発足後

国鉄の分割・民営化、つまりJR発足後、東北・上越新幹線はJR東日本、東海道新幹線はJR東海、山陽新幹線はJR西日本の運営とされたが、当初設備は第3種鉄道事業者の「新幹線保有機構」が保有し、各会社が第2種鉄道事業者として路線を借り受けて運営する形とした。新幹線の保守費用は各社が負担し、新幹線保有機構は設備の貸し代だけを受け取るもので、利益の出る新幹線事業によって赤字となる他地域JR会社への補填を行うのが目的であった。

しかし、前記JR3社の経営が安定化して、東京証券取引所などへの上場が視野に入ると、輸送量に応じて貸し賃が変わるこの制度のままでは会社の営業努力が反映されないことや、各社の資産・債務の額が確定できないことなどが問題視され、結局1991年（平成3年）に制度を変更し、各鉄道会社が新幹線資産を新幹線保有機構を改編した鉄道整備基金から60年賦で買い取ることにした。

分割・民営化後、技術・営業面で停滞していた新幹線も新型車両の登場、新形態など積極的な流れが見られるようになった。

後者の代表として、JR東日本は新幹線規格（フル規格）の線路を新規に建設することなく、既存の在来線を改良し、専用の車両を新造したうえで、新幹線と在来線が直通運転できるようにしたミニ新幹線を整備した。

1992年（平成4年）に400系を新造し、山形新幹線として奥羽本線の福島駅 - 山形駅が、1997年（平成9年）にE3系を新造し、秋田新幹線として田沢湖線・奥羽本線の盛岡駅 - 秋田駅が、1999年（平成11年）にE3系1000番台を増備し、山形新幹線の延伸として奥羽本線の山形駅 - 新庄駅が、それぞれ順次営業運転を開始した。

JR西日本は山陽新幹線博多総合車両所への車庫線を旅客線化し、1990年（平成2年）に博多南線として博多駅 - 博多南駅を、こだま号に使用される車両を用いる在来線特急という形態で営業運転を開始した。

また最高速度は210 km/hの時代が長く続いたが、国鉄末期頃（→1985年3月14日国鉄ダイヤ改正・1986年11月1日国鉄ダイヤ改正も参照）から次第に向上されるようになり、21世紀初頭では東海道新幹線で270 km/h、山陽新幹線区間で300 km/h、東北新幹線区間で320 km/hに至っている。また時速アップ以外にも、停車駅での停車時間の短縮や、停車駅間の速度をできるだけ高速度で維持するなどして、わずかな分単位ながら、主要駅間の時間短縮を図る工夫もされている。

国鉄末期に建設が凍結されていた整備新幹線は工事が再開され、東北新幹線（2002年に八戸まで、2010年に新青森まで延伸）と九州新幹線鹿児島ルート（2004年に鹿児島中央 - 新八代間、2011年に新八代 - 博多間が開業）は既に全線開業、北陸新幹線（1997年に「長野新幹線」として長野まで開業、2015年に金沢まで延伸、2024年に敦賀まで延伸開業）と北海道新幹線（2016年に新函館北斗まで開業）と西九州新幹線（2022年に長崎 - 武雄温泉間が開業）が部分開業し、残った区間も工事が次第に進みつつある。

また20世紀末以降、新幹線による通勤・通学が増加した（「新幹線通勤」も参照）。これは、いわゆるバブル時代の大都市における地価の高騰で、新幹線で通勤・通学が可能な郊外（主に東京への通勤・通学を目的に栃木県、群馬県、静岡県東部が多い）の住宅に住む人が増えたためである。1983年（昭和58年）2月の新幹線定期乗車券販売開始をきっかけに、新幹線通勤定期券を支給する企業の増加、さらに企業が支給する通勤定期券代の所得税非課税限度額の引き上げがそれに輪をかけた。朝・夕の新幹線においては通勤客で混雑が激しくなり、通勤客向けのダイヤも設定されるようになった。これに対応してJR東日本ではMaxという多座席型の2階建車両を投入し、1列車あたりの定員を大幅に増やした時期もあった。首都圏以外でも、山陽新幹線の小倉 - 博多間などで通勤・通学に新幹線を利用する者が多くなった。
安全性

1964年（昭和39年）10月1日に最初の新幹線である東海道新幹線が開業して以来、事業者側の責任事故として確定した事故は、1995年（平成7年）に駆け込み乗車の乗客の手をドアに挟んで引きずり死亡させた三島駅乗客転落事故（旅客が死亡）と、2015年（平成27年）に発生した山陽新幹線部品脱落事故（旅客が負傷）の計2件である。

駅ホームでの事故（駅ホームから転落または故意に投身し車両に接触、あるいは架線に接触し感電など）や線路内立ち入り等による死亡例は多数発生しているものの、これらは鉄道事業者側の責任事故ではなく、またこれらは新幹線システムそのものの根本的欠陥に起因する事故ではないため、新幹線の安全性に関しては非常に高いものと捉えられている。この事実は新幹線の安全神話などと称されていた[52]。

2011年には「最も安全な高速鉄道ネットワーク」としてギネス世界記録に認定される[53]など、新幹線車両自体の脱線、転覆や衝突による旅客死亡事故は未だ発生していないが、重大な事故に至る一歩手前の事態は過去に何度か発生している。
事故の事例
	
出典は列挙するだけでなく、脚注などを用いてどの記述の情報源であるかを明記してください。記事の信頼性向上にご協力をお願いいたします。（2015年6月）

以下の各事例は、新幹線の安全を脅かす事故例と考えられ、重大視されてきた。

    1973年（昭和48年）：東海道新幹線大阪運転所脱線事故
    1974年（昭和49年）：東京運転所（品川基地）分岐線と新大阪駅構内で相次いで発生したATC異常信号事故
    1991年（平成3年）：ひかり291号（100系X編成）が、三島駅まで車輪を固着させたまま走行した事例
    1997年（平成9年）：山陽新幹線の岡山新幹線運転所内で過走して脱線した事故[54]
    1999年（平成11年）：山陽新幹線福岡トンネルコンクリート塊落下事故
    2010年（平成22年）：山陽新幹線広島-新神戸間において、「のぞみ56号」の台車のギヤボックスカバーが破損、漏れた油が過熱し車内に白煙が充満、乗客1名が負傷した事故[55]
    2015年（平成27年）：山陽新幹線小倉 - 博多間において、「さくら561号」のカバーが脱落、脱落したカバーが車体を損傷させ、乗客1名が負傷した事故
    2017年（平成29年）：のぞみ34号（N700系K5編成）が、名古屋駅まで台車に亀裂が入った状態で走行した事例（新幹線の事故で初めて国交省運輸安全委員会から「重大インシデント」認定）

他にも1990年代末期から多発したトンネルのコンクリート剥落事故に対しては、JR各社には設備保全を徹底させる対策が求められている。

また在来線と直通運転を行うミニ新幹線（山形新幹線・秋田新幹線）では在来線区間に踏切もあるため、踏切事故がしばしば発生している。
地震に伴う安全への脅威

地震に伴う障害は、高速鉄道の安全性にとって脅威となりうる。このため、地震が多発する日本において運営される新幹線にとっても課題となっている。1990年代以降、日本国内における大きな地震災害の多発により、高速鉄道の地震に対する脆弱性が指摘されるようになった。
大井川河口を震央とする地震の事例

1965年（昭和40年）4月20日、静岡県の大井川河口を震央とするマグニチュード6.1の地震が発生し、開業後約半年経過していた東海道新幹線にも静岡市周辺の盛り土が崩れる被害が生じた。当時はまだ運転本数が1時間に片道2本しかなく、発生直後にすべての列車の運行が停止されたこともあり、走行中の車両や旅客には大きな被害は発生しなかった。しかし、当時の運行責任者であった斎藤雅男（元国鉄新幹線支社運転車両部長-新幹線支社次長、鉄道工学の専門家）によると、「当時は雨の影響で地盤が弱くなっており、大きく陥没していたところもあった。仮に崩れた路盤上に列車が来ていたら間違いなく脱線して大惨事になっていた」という。なお、山陽新幹線の一部区間と東北新幹線以後の新幹線にはスラブ軌道が採用されている[56]。
兵庫県南部地震の事例

1995年（平成7年）1月17日の兵庫県南部地震（阪神・淡路大震災）では、被災地域において山陽新幹線の高架橋が破損・一部落下し、新大阪駅 - 姫路駅間が81日間にわたり不通となった。地震発生は午前5時46分で、始発列車は新大阪駅に停車していたがこの日の営業運転が始まる前であったため、落下した高架橋に列車が突っ込むなどの最悪の事態は免れた。これを機に高架橋の補強などの耐震対策が進められた。また比較的被害の少なかった京都駅 - 新大阪駅間、姫路駅- 岡山駅間は1週間程度で復旧した。
新潟県中越地震の事例
詳細は「上越新幹線脱線事故」を参照

さらに、2004年（平成16年）10月23日の新潟県中越地震においては、上越新幹線が甚大な被害を受けた。高架やトンネルなどの構造物に損傷が発生したほか、上越新幹線列車の「とき325号」（10両編成、200系=K25編成）が長岡駅の手前付近を約200 km/hで走行中に脱線した。これはまた新幹線史上初の営業運転中、しかも高速走行中の脱線事故となった。この脱線の衝撃で、レールの道床の締定が多数外れ、一部のレールはねじ曲がるなどの大きな被害を受けた。

通常、列車がこの規模の地震に震源地付近で直撃された場合、たとえ停車していたとしても脱線は免れ得ないと考えられる。「とき325号」の事例では約200 km/hで脱線したとはいえ、奇跡的に、死者・重傷者などは生じなかった。これは、編成全体の横転などには至らなかったこと、および、数分の差で対向列車との衝突も免れるという幸運も重なったことによる。なお、横転が生じなかった理由には、事故現場が積雪の多い地帯であるため、レール脇に雪を融かして流すための溝があり、そこに脱線後の車体の一部がはまり込んだことも関係していたとされる。

なお、地震に対する脅威に対し、上越新幹線にも地震感知システム「ユレダス」をカスタマイズした「コンパクトユレダス」が採用されている。実際、この「とき325号」のケースにおいても初期微動（P波）の検知後にブレーキが動作した。しかし、このケースでは、「とき325号」の通過地点からみて直下型の地震であったため、「とき325号」の停止前に地震が到達したことにより被害が生じてしまった。

この事故により、新幹線を運営するJR各社は、新幹線における地震対策の重要性を強く認識することとなった。
東北地方太平洋沖地震での事例

2011年（平成23年）3月11日に発生した東北地方太平洋沖地震（東日本大震災）では、東北新幹線が甚大な被害を受けた。運転中の列車はすべて強制停止したが、大宮駅 - いわて沼宮内駅間の広範囲にわたって、高架橋の損傷や架線柱の倒壊などが発生した。仙台駅では試運転中のE2系の一部車両が脱線したほか、ホームの天井板が剥がれ落ちるなど前記の中越地震以上の被害を受け、全線復旧まで約1か月半を要した。
熊本地震での事例

2016年（平成28年）4月14日に発生した熊本地震では、九州新幹線が被害を受けた。下りの800系6両編成の回送列車が熊本駅から熊本総合車両所へ向かう途中に脱線したほか、防音壁の落下や高架橋の亀裂などが見つかったが、13日後の4月27日午後には全線で運転を再開した。
福島県沖地震での事例
詳細は「東北新幹線脱線事故」を参照

2022年（令和4年）3月16日に宮城県牡鹿半島沖南南東60kmを震源とする最大震度6強の地震（福島県沖地震）のため東北新幹線「やまびこ223号」が福島駅 - 白石蔵王駅間の宮城県白石市内で営業運転中に脱線した。現場付近及び新幹線の車両には脱線を防ぐ装置が設置されていたものの17両編成のうち16両が脱線し、合わせて68の車輪の軸のうち60基がレールから外れていた[57]。また、脱線した列車には「レール転倒防止装置」や「逸脱防止ガイド」が設置されていたが一部の車両はレールから大幅にずれて傾くなどの被害が確認された。

「やまびこ223号」が脱線した福島駅 - 白石蔵王駅の間は、最高速度が320 km/hとなっていたが、地震が起きた際列車は白石蔵王駅での停車に向け減速していた際に発生し、地震を検知して非常ブレーキがかかる過程で脱線したと見られている[58]。初期微動（P波）を検知して自動でブレーキを掛けさせる装置は正常に作動していたという[59]。

営業運転中の新幹線が地震による脱線事故を起こしたのは新潟県中越地震での上越新幹線脱線事故以来2例目である[59]。
災害・テロへの対策不足

新幹線では、航空機や船舶と異なり、通常の運行では乗客名簿などは整備されない。万一、転覆事故などで多数の死傷者が生じた時には、死傷者の身元特定に支障をきたすのではないか、との指摘もある。もしそうなった場合、家族への連絡や事故の補償などで大きな問題となることが予想されるが、新幹線を運営する各鉄道会社はこの課題について踏み込んだ対策を採るまでには至っていない。

20世紀末から世界的に増加しているテロリズムに対しても新幹線は脆弱ではないかとの指摘もされている。現状では航空機のような搭乗時の手荷物検査がなく、その気になれば車内やプラットホームに、可燃物や爆発物、刃物を容易に持ち込むことができるのも事実[注 9]である。また高架橋などの軌道設備には周囲から容易に接近できる箇所が多く、この面でもたやすくテロの対象となりうる。

なお、N700系では全てのドア上部と乗務員室出入口に、E5系・H5系やE6系、E7系・W7系ではそれらに加えてグランクラスを除く客室内にも監視カメラを取り付けている。東海道新幹線車内での放火事件以降、JR東海はN700系でも客室内とデッキ通路部にも増設することとし[60]、2016年2月23日より増設が完了した編成が運行を開始した（1編成あたりのカメラは60台から105台に増加）[61]。なお、客室内の映像については、N700系は常時録画だが、その他のE5系・H5系などはプライバシー保護の観点から非常ボタンが押されてから録画を開始する方式としている。JR東日本ではこれらの車両についても今後は同様に常時録画とする方針である[61]。

JR東日本の各新幹線では、テロ対策のため、車内のゴミ箱を一切利用停止にしていた時期があったが、乗客からの不便という声が高まり、現在は利用を再開している。ただし要人来日時（特に主要国首脳会議開催時やアメリカ合衆国大統領来日時）は、利用を停止する場合がある。また、JR東海ではアメリカ同時多発テロ事件以降、系列の警備会社と連携し、沿線を24時間体制で巡察しているほか、全ての列車に警備員を警乗させている。
救命対策

2008年（平成20年）7月28日に東海道・山陽新幹線では、2008年12月より全編成において自動体外式除細動器 (AED) を配備すると発表した[62]。
世界への影響
世界の高速鉄道の最高速度
「高速鉄道」も参照
超伝導リニア L0系。2015年4月に山梨実験線にて世界最高速度603 km/hを記録

世界初の210 km/h運転を達成した新幹線の成功は、欧米各国に影響を及ぼした。鉄道先進国を自負していたフランスは、1967年5月28日よりパリ - トゥールーズ間の列車「ル・キャピトール」を欧州において初めて[注 10]一部区間で200 km/hで運転し、その後も複数の列車を200 km/hで運行していた。新幹線の開業後、1981年に本格的な超高速列車TGVを開発し、営業最高速度260 km/hというスピード世界一を達成し、新幹線の記録を凌駕した。

その他、ドイツ (ICE) やイタリア（ディレッティシマ）でも高速列車が計画され、実現に移された。イタリアのディレッティシマは欧州初の高速新線であり、1970年に工事が始まり、1978年に部分開業を迎え、1983年に250 km/h運転を開始したものの、その後の整備で仏独に遅れを取り、全線が開業したのは1992年である。

スペインでは、当初はイベリア軌間の在来線を高速化させる計画で進めていたが、1986年に同国が欧州諸共同体 (EC) へ加盟したのちに標準軌の高速新線を導入する計画へ方針転換。この一連の流れにおいては日本企業が車両の導入に大きな努力をしており、技術面において非常に高い評価を得ていたが、同国の欧州諸共同体への加盟や同国への投資額、同国内の経営不振企業の救済計画などに代表される政治的な問題からフランスのTGV方式の高速列車を採用することとなり、この時点での日本の新幹線方式の車両の導入は幻となった。

ロシアの高速列車ソコルは1997年、ドイツ鉄道の技術支援を受け、モスクワ - サンクトペテルブルク間654 kmを営業最高速度250 km/hで結んだことにより、所要時間が、4時間20分から2時間30分に短縮された。

なお、すでに標準軌の鉄道網が整備されているこれらの国では、駅周辺は従来の路線をそのまま使用し、郊外区間では諸条件によって高速新線建設と在来線改良を使い分けることが多く、全線を新線として建設する新幹線とは異なっている。

走行試験を除く営業運転速度は、2019年4月現在、中国高速鉄道のCR400系（京滬高速鉄道と京津都市間鉄道）の 350 km/hが世界最速である。新幹線E5系やフランスTGV (320 km/h) などがそれに続く。大韓民国（韓国）では2004年にフランスのTGV方式の韓国高速鉄道 (KTX) が300 km/hで開業し、台湾では2007年に日本の新幹線方式（一部仏独の技術を導入）の台湾高速鉄道が300 km/hで開業した。

TGVは360 km/hへの速度向上を計画している。2007年後半からフランスのTGV方式を採用したスペインのAVEは、マドリード - バルセロナ間630 kmの新線で、ドイツのICEの技術に使われているシーメンス製の Velaro E という列車を使い350 km/hで運転する計画がある。実現すれば、マドリード - バルセロナ間は2時間30分に短縮される。 さらにロシアやベトナムでも新幹線をモデルにした最高速度350 km/hの高速鉄道建設が計画されている。

浮上式鉄道を含めた2015年4月時点における世界最速の旅客営業鉄道路線は、2003年にドイツの技術によって開業した、中国・上海浦東国際空港へのアクセス路線である上海トランスラピッドで、その最高速度は430 km/hである。 走行試験も含めた鉄道における最高速度の世界一は、日本のL0系が2015年4月に山梨リニア実験線で記録した603 km/h[63]。浮上式鉄道を除くとフランスTGVの高速試験車V150編成が記録した574.8 km/hである（日本の非浮上式鉄道の最高記録はJR東海の300Xによって達成された443 km/hで世界第3位）。

東海道新幹線は建設時期が古く、カーブなどの線路状況が200 km/h台の設計になっている。より新しい山陽新幹線・東北新幹線などもフランスやドイツなどと比較すると山岳区間が多く、路線の起伏やカーブの設計などにおいて高速化を妨げる点が多い。特に後者は上越新幹線とともに寒冷地の耐寒・耐雪装備が不可欠であり、重量的に不利である。また沿線に住宅地が多いため、騒音への対策も必要となるなど、300 km/h以上の運転には課題が多い。

しかし、JR東海・JR西日本は2007年より山陽新幹線で500系と同じ最高速度300 km/h、東海道新幹線でも従来の車両では255 km/hまで減速する必要のあった半径2,500 mのカーブを、車体傾斜装置を搭載することで270 km/hで通過できるN700系の導入を開始した。さらに改良型となるN700系1000番台（通称「N700A」）の製造及びN700系からの同仕様への改造により、東海道新幹線で285 km/hでの営業運転を2015年3月14日より開始した。これにより、半径2,500mのカーブを、275 km/hで通過するようになった。また、JR東日本は2004年から360 km/h走行を前提とした試験車両（E954・E955形）を開発し、2009年からはE954形をベースとして320 km/hでの走行を前提にしたE5系を製造、新青森延伸後の2011年3月5日に300 km/hで営業を開始し、2013年3月16日より320 km/hでの営業運転を開始した。

かつて、JR東海は東海道新幹線の一部区間で、当時の営業最高速度270 km/hから330 km/hに引き上げることを検討していた。330 km/hでの走行は京都 - 米原間の直線が長い一部区間を対象に、N700系を使用し、前方に待機列車がなく性能を存分に発揮できる「のぞみ」の始発、終発に限定して行う計画であった。

また近年、速度向上の動きが顕著である。JR東日本はE954形-E5系の開発時に断念した360km/hでの営業運転について再度挑戦すべく、試験車両E956形を用いて走行試験を開始した。また、JR北海道は建設中の新函館北斗駅 - 札幌駅間について、整備新幹線としては初となる320km/hで営業運転するための追加工事を行うことを発表した。さらに、JR東海はN700Sにて営業用車両としては国内最速である363km/hを記録した（この速度で営業運転するためではなく、国内外に高い走行性能を示す目的）。
浮上式鉄道を除く、営業運転での最高速度記録
新幹線以外の高速鉄道の詳細な記録は「高速鉄道の最高速度記録の歴史」を参照
また、東海道・山陽新幹線と東北・上越・北陸新幹線では「最高速度」の考え方に差異があることに注意。「自動列車制御装置」のATC-1型、ATC-2型を参照

各路線での記録である。

    1964年10月1日：東海道新幹線開業。210 km/h。軌道の安定を図るため一部区間で160 km/h。
    1965年11月1日：東海道新幹線の徐行運転が解除され、全線で210 km/h。
    1972年3月15日：山陽新幹線開業。210 km/h。
    1981年9月27日：フランスでTGV・LGV南東線が開業。260 km/h。
    1982年
        6月23日：東北新幹線開業。210 km/h。
        11月15日：上越新幹線開業。210 km/h。
    1983年9月25日：フランスでTGVが270 km/h。
    1985年3月14日：東北新幹線で240 km/h。
    1986年11月1日：東海道・山陽新幹線で220 km/h。
    1988年3月13日：上越新幹線で240 km/h。
    1989年
        3月11日：山陽新幹線で230km/h。
        9月24日：フランスでTGV・LGV大西洋線が開業。300 km/h。
    1990年3月10日：上越新幹線で275 km/h。大清水トンネル下り坂利用。
    1992年3月14日：東海道新幹線で270 km/h。
    1993年3月18日：山陽新幹線で270 km/h。
    1997年
        3月22日：東北新幹線で275 km/h。山陽新幹線で300 km/h。
        10月1日：北陸新幹線開業。260 km/h。
    1999年12月4日：上越新幹線、275 km/h運転を廃止し、240 km/hに変更。
    2002年8月1日：ドイツでケルン-ライン＝マイン高速線が開業。ICE 3で300 km/h。
    2004年
        3月13日：九州新幹線鹿児島ルート開業。260 km/h。
        4月1日：韓国で韓国高速鉄道 (KTX) が開業。300 km/h。
    2007年
        1月5日：台湾高速鉄道が開業。300 km/h。
        6月10日：フランスでTGV・LGV東ヨーロッパ線が開業。320 km/h。
    2008年8月1日：中国で京津都市間鉄道が開業。350 km/h。
    2010年10月26日：中国で滬杭旅客専用線が開業。350 km/h。
    2011年
        3月5日：東北新幹線で300 km/h。
        7月1日：中国・京津都市間鉄道が300 km/hに変更。
        8月28日：中国・滬杭旅客専用線が300 km/hに変更。
    2013年3月16日：東北新幹線で320 km/h。
    2015年3月14日：東海道新幹線で285 km/h。
    2016年3月26日：北海道新幹線開業。260 km/h。
    2017年9月21日：中国・京滬高速鉄道で350 km/h。

新幹線の輸出
台湾
詳細は「台湾高速鉄道」を参照

中華民国（台湾）の台北 - 高雄間のうち、台北 - 左営の約340 kmで運行されることになった高速鉄道路線（台湾高速鉄道）は、独仏連合が先行する形で日本連合との間で熾烈な受注競争が行われたが、1999年に台湾で大地震が起き[64]、地震時のリスクを台湾側は懸念するようになり、その結果新幹線は地震時に安全性が高いことに焦点が当たり、日本連合が最終的に逆転、車両や運行管理システムなど根幹部分の受注に成功した[64]（ただし、台湾高速鉄道の全体を完全に日本連合だけが受注したわけではない ※）。車両には700系をベースとした700T型が用いられ、運行管理システムも新幹線のもの、乗務員のトレーニングも日本式のものが採用された。台湾高速鉄道の顧問には、日本における新幹線計画の実現に大きく貢献した島秀雄の次男である島隆が就いた。一方、鉄路部分の施工に関しては、競争で先行した欧州連合がすでに契約締結済で関与し、韓国も関与した。

当初は2005年10月の開業を目指して建設が進められたが、台湾高速鉄道のコンサルタント業務を欧州連合が先に受注していたため、施工方法やスケジュールの調整が難航。また建設工事の一部区間を受注していた韓国の現代建設による路盤の手抜き工事が発覚するなど、各国企業の思惑が入り乱れたため、開業時期が徐々に遅れ、結局2007年1月5日に板橋 - 左営間で仮開業し、全線は2007年3月に正式開業した。2016年に、台北 - 南港間が開業しているが、左営 - 高雄間の着工は未定である。
アメリカ合衆国

アメリカ合衆国・テキサス州ヒューストン - ダラス間を90分で結ぶテキサス・セントラル・レイルウェイの建設計画が日本の新幹線方式を導入することが決まり、早ければ2021年に建設を開始し、2026年の開業を予定している[65][66]。なお既に日本の新幹線方式で建設し、N700系の改良版を導入することを明言している。JR東海が技術支援に加え、少額の出資を検討している[67][68]。
イギリス

日立製作所は、2009年からロンドン - ケント州間の高速新線「High Speed 1（HS1、旧名CTRL: Channel Tunnel Rail Link）」で運行される高速列車サービス「オリンピックジャベリン」の専用車両「クラス395」29編成計174両を受注し、2007年8月から引き渡しが始まった。車両はHSBC Rail UKが保有し、サウスイースタンが列車の運行を担当する。UIC規格路線を走る初めての日本製高速鉄道車両であり、HS1上においてTGVベースのユーロスターと混在して運行されることとなる。営業最高速度は、HS1上で140 mph (225 km/h)、在来線では70 mph (112 km/h) で、将来的には170 mph (275 km/h) を目指す。

日立はさらなる高速鉄道事業の受注を目指して、都市間高速鉄道計画における受注も目指して活動を展開し、2009年2月に、イースト・コースト本線（ロンドン - エディンバラ間、距離700 km）やグレート・ウェスタン本線（距離300 km）を走行する高速鉄道車両の製造の優先交渉権を得た。そしてキャメロン政権の歳出削減政策による一時の交渉凍結を経て、2012年7月にシーメンス、アルストム、ボンバルディアの鉄道ビッグ3との受注競争に打ち勝って、596両の高速鉄道車両の製造と2017年から27年間にわたる車両のリースと保守事業を、45億ポンド（受注時の為替レートで約5,500億円）で一括受注した。さらに2013年7月には、追加で「クラス800」270両の製造と27年間にわたる車両のリースと保守事業を12億ポンド（受注時の為替レートで約1,800億円）で一括受注した[69][70]。受注した車両は最高速度200 km/hで、5両編成（定員約300人）と9両編成（定員約600人）で運行される。これを受けて日立は8000万ポンド（96億円）を投じて英北部ダラム州のニュートン・エイクリフに車両生産工場を建設し、2016年から月産35両のペースで車両を生産する。車両のリース事業は日立の子会社などが出資する特別目的会社アジリティトレインズによって行われる。
中華人民共和国
詳細は「中華人民共和国の高速鉄道」および「中国高速鉄道CRH2型電車」を参照

中華人民共和国は京滬高速鉄道など8路線、計7,000 kmの旅客専用線（最高速度350 km/h）のほか、中国全土に高速鉄道網の建設（最高速度200 - 350 km/h）を進めているが、国産高速車両「中華之星」の開発で多くのトラブルに見舞われたこともあって、日本の新幹線の技術導入と同時にフランス・ドイツ・カナダなどからも技術導入を図っている。

2007年には、主要都市間の在来線高速化 (200 – 250 km/h) に向け、JR東日本のE2系（川崎重工業製）をベースにしたCRH2型「子弾頭」を導入している。日本以外ではスウェーデンのRegina（カナダ・ボンバルディア製）をベースにしたCRH1型（CRHは“China Railway High-Speed”の略）、イタリアのペンドリーノETR600（フランス・アルストム製）をベースにしたCRH5型も導入された。

2008年の夏季オリンピックに合わせて開業した北京 - 天津間の京津都市間鉄道にはCRH2型の旅客専用線仕様も導入されたが、設計速度を超過し運転していたため、日本側からの抗議を受けて他線区に転配し、武広線では250 km/hで運転している。同年12月からは、北京 - 上海・杭州間にCRH2E型の16両編成中13両を寝台車「軟臥車（B寝台相当）」とした夜行列車が運転されている。「夜行新幹線」はかつて日本でも試作車両が作られたが、営業運転は実現していない。

2009年に開業した武漢 - 広州間の武広旅客専用線 (350 km/h) にはドイツのICE 3（シーメンス製）をベースにしたCRH3型が導入されている。どの国からも、一部は完成車で納入され、残りは現地組み立てまたは技術供与による現地生産となっている。

なおJR各社では、JR東日本が受注に積極的なのに対し、台湾への技術供与を行ったJR東海会長の葛西敬之は、法整備が不十分な中国においてトラブルが発生した場合の責任問題や、中華人民共和国側の車両購入条件である「中華人民共和国へのブラックボックスのない完全な技術供与」では技術流出の危惧から反対の意見を表明している。また、2010年4月に葛西が「中国の高速鉄道は安全性を軽視することで、限界まで速度を出している。技術も『外国企業から盗用』」と主張したことに対し、中国の何総工程師は「我々が求めている技術は、日本のような島国向けの技術とは異なる」と主張し、「安全性が保証されている中国の高速鉄道技術は既に世界をリードする地位を獲得した」などと反論した[71]。

しかし、中国鉄道部科学技術局長などを務めた周翊民は、中国紙『21世紀経済報道』に対し「世界一にこだわり、設計上の安全速度を無視し、日独が試験走行で達成していた速度に近い速度での営業を命じただけで、中国独自の技術によるものではない」と暴露し、「自分の技術でないので問題が起きても解決できない。結果の甚大さは想像もできない」と指摘した[72]。またアメリカ議会の超党派諮問機関である「米中経済安保調査委員会」は2011年10月26日、日本の新幹線技術の中国側の取得について「中国企業が外国技術を盗用した最もひどい実例」と明記し、中国の政府や国家の意思によるものだと結論付けた[73]。なお、2011年7月23日に浙江省温州市で、死者43人・負傷者190人以上を出す追突脱線事故が発生し、鉄道局長ら幹部3人が更迭されている[74]。
輸出候補地

    タイ王国の旗 タイ
        バンコク - チェンマイ間(670 km)を結ぶタイ高速鉄道計画に新幹線を採用することを、日本・タイ政府間で2012年から検討してきたが、2015年5月27日に新幹線方式導入を前提に調査を始める覚書を締結[75][76]。事業費は1兆円超を見込む[77]。2020年にも開通する見通し[78]。タイでは、中華人民共和国も一部路線を受注しており[79]、競争が激化している。
     ベトナム
        ハノイ - ホーチミン間 (1,630 km) を最高速度350 km/hで結ぶベトナム高速鉄道計画があり、完成すれば現在30時間以上かかっている所要時間が10時間弱に短縮されると期待されている。資金は日本の政府開発援助（円借款）を充てる予定で、新幹線方式の導入が検討されていたが、2010年6月19日のベトナム国会でこれらの政府案は否決され、暗礁に乗り上げた格好となった。
    インドの旗 インド
        2007年5月、インド国鉄はインド高速鉄道計画事前事業化調査のための説明会を開催し、日本やヨーロッパの車両メーカーも参加した。計画はアフマダーバード - ムンバイ間、アムリツァル - ニューデリー - ラクナウ間、パトナ - コルカタ間、チェンナイ - バンガロール間の4路線[80]。このうち、ムンバイ - アーメダバード間(500 km)について、2017年着工、2023年開業予定。費用は9800億ルピー（147億米ドル）で、日本から最大で1兆4600億円の円借款で資金調達することが2015年12月の日印首脳会談にて合意された[81]。
    アメリカ合衆国の旗 アメリカ合衆国・カリフォルニア州
        ロサンゼルス - サンフランシスコ間などを結ぶカリフォルニア高速鉄道の建設計画がある。州の予算や採算性の問題もあり建設時期は未定のままだが、具体的に進行し始めている。JR東海が積極的に新幹線を売り込んでいる。アメリカ西部は地震も多く、開業以来地震に対する対策を採ってきた新幹線はその点で各国の高速鉄道よりもアドバンテージがあるのでは、とする声もある。建設プロジェクトを紹介するインターネットのウェブサイトには、700系新幹線をイエローとブルーのツートンカラーにした車両のCG動画が公開されている。また、JR東日本も新幹線を売り込んでいる見込みである。
    ブラジルの旗 ブラジル
        リオデジャネイロ - サンパウロ - カンピーナス間にブラジル高速鉄道計画（路線距離550 km、最高速度320 km/h）がある。当初は2014年にブラジルで開催されるワールドカップに合わせて開通させたい意向で、日本は国土交通省と三井物産、三菱重工業、川崎重工業、東芝が官民共同で売り込んでいたが[82][83]、提示条件の厳しさなどから入札の不調や延期を繰り返している。リオデジャネイロ - カンピーナス間は標高差が700メートル近くあり、長野新幹線の高崎駅 - 軽井沢駅間などの大きい標高差における建設、運行のノウハウを蓄積している新幹線は、その点で各国の高速鉄道よりも優位ではないかと評価する声もある。
    ロシアの旗 ロシア
        モスクワ - サンクトペテルブルク高速鉄道運行プロジェクト（路線距離645 km、最高速度350 km/h）が進行中であり、同プロジェクト一行が日本企業と接触している。2009年12月からは同区間の在来線にICE3ベースの新型車両が導入され、最高速度250 km/hで運転される予定。さらにモスクワ - ニジニ・ノヴゴロド間、エカテリンブルク - チェリャビンスク間などにも高速新線の建設が計画されている。モスクワ - ウラジオストック間を結んでいる路線距離世界一 (9,288 km) のシベリア鉄道についても、一部高速新線（最高速度350 km/h）の建設を含む近代化計画がある。
    インドネシアの旗 インドネシア
        インドネシアでは経済開発を加速する目的で、2008年に「経済開発加速化・拡充マスタープラン2010 - 2025（MP3EI）」を策定[84]。このプランの一部として、ジャカルタから西部に約700 kmにあるスラバヤまでのジャワ島東西を結ぶ路線と南東に約150 kmあるバンドンまでのインドネシア高速鉄道計画がある[84]。2011年には「国家鉄道整備総合計画」が策定され、ジャカルタ - スラバヤ間の高速鉄道が位置付けられている[84]。2030年をめどに開業を目指すとしている[84]。
        日本はインドネシアのインフラ整備として、高速道路や発電所建設とともに、新幹線を売り込んでいる。しかし、インドネシアは高速道路や発電所と比べると高速鉄道は優先度が低いとして、新幹線の導入については慎重な姿勢である[85]。2015年9月3日、インドネシア政府は高速鉄道計画の撤回を発表して入札を白紙化したが、9月29日に財政負担を伴わない中華人民共和国国案の採用を決定した[86][87]。2023年10月開業した。[88]
    マレーシアの旗 マレーシア
        ナジブ・ラザク首相は、2015年5月25日、日本経済新聞の取材に対して、「日本の新幹線は安全性や信頼性などで競争力が高い」として、候補の一つであることを明らかにしている[89]。

その他
アメリカ合衆国・北東回廊

アメリカ合衆国の北東部には、ワシントンD.C.、フィラデルフィア、ニューヨークシティ、ボストンなど名だたる大都市が連なり、メガロポリスと呼ばれる。それらの都市は1800年代に建設された北東回廊と呼ばれる幹線鉄道が結んでおり、その特性から同国で最も利用者の多い幹線鉄道路線となっている。この北東回廊のうち、ワシントンD.C.とニューヘイヴン間は1900年代前半に電化が完了、電気動力による高速運転が実現しており、1960年代後半には東海道新幹線の成功に影響された電車高速列車メトロライナーが登場。しかし、度重なる運営事業者の経営不振と再編により軌道保守へ十分な投資ができず、せっかくの高速列車の乗り心地を悪くさせ、速度を制限させる原因となっていた。その状況を打破すべく、1971年に民営企業より列車運行および線路保有権を引き継いだ全米旅客鉄道公社 (Amtrak) は北東回廊の軌道を強化させ、同時にメトロライナー電車よりも高速な列車の導入を決定。この一連の流れで、新幹線の成功を経験していた日本はこの計画に参加することとなり、軌道強化計画のほか、新幹線0系を基礎としつつもアメリカ合衆国の基準に適応させた専用車両の設計を行った。

この時期、二度にわたる石油危機などの影響で、折しも鉄道利用者が増加していたという背景も追い風となり、北東回廊の更新計画は北東回廊改良計画として1980年代に実を結んだ。しかし、最も重要な課題であった軌道強化には日本の技術が大いに活かされたものの、初の新幹線仕様車両の国外輸出として期待されていた専用車両の導入が実現することはなかった。
アメリカ合衆国・オハイオ州

1970年代後半のアメリカ合衆国。同国の中西部に位置するオハイオ州では、州内の各都市を高速で結ぶ新たな鉄道を建設する話が盛り上がった。この当時、ほかに例を見ない高速鉄道として有名であった日本の新幹線はこの計画の恰好のモデルとなり、日本国鉄およびその関連機関はこの計画に対して様々な支援を行ったが、実現していない。
大韓民国

大韓民国の高速鉄道・"KTX"計画において、日本は新幹線方式（車両は300系電車を基礎としたもの）で入札に参加した。しかし、「反日感情」などの政治的な理由により、最終的にフランスのTGV方式の導入となった。
スペイン

三菱グループを筆頭とした日本連合がスペインの高速鉄道、AVEの第一期区間（マドリード＝セビージャ高速線）で運行される車両の入札に参加し、日本の新幹線100系を基礎とした車両を提案[90][91]。ヨーロッパ各国の名だたるメーカーとの競争に勝ち、フランスのTGV、ドイツのICEとともに最終選考に進み、その中でも価格・経験の面でもっとも高い評価を得たが、スペインの欧州諸共同体（EC）への加盟などが影響し、フランスのTGVに敗れる結果となった[90][91]。
イラン

1970年代、イラン政府は日本政府に対し、同国の首都テヘランと同国第二の規模を誇るマシュハドの間に新幹線の建設を行うように要請。実際に路線経路の計画や車両設計の準備などが行われたものの、1979年の革命で新幹線を要求していた政府が打倒されたことにより、計画ごと消滅した。
他国との競合点

    2009年現在、世界における高速鉄道の技術入札においては、事実上、日本の新幹線技術とフランスのTGV技術の二大勢力が競合している[92]。
    世界的に見ると、高速鉄道を必要とする国には、日本のように地理的条件や騒音対策・輸送量の面で過酷な条件に置かれている例はさほど多くはないため、新幹線方式よりもコスト面でより有利な、TGVに代表される半動力集中式[要出典]を採用する場合が多い。
    競合点のひとつである輸送量については、新幹線は同じ標準軌のTGVなどより車両の幅が広く高頻度運行が可能であるため、単位時間あたりの最大輸送量も大きい。
    アメリカにおける高速鉄道入札でのプレゼンテーションにおいては、日本側は安全性その他の優位性を主張しつつ、線路・システムなどとの一括での契約を要求したのに対し、フランスのTGVは車体のみでの契約も可能とするなどより柔軟な交渉をしている[92]。

貨物輸送
交通博物館で展示されていた東海道新幹線貨物列車（電車）の模型
新幹線大阪運転所（鳥飼基地）の京都側にある本線をまたぐ構築物と合流跡

新幹線による貨物輸送は、最高速度や制動距離などの違いからダイヤグラム上で旅客列車と混在させることは現状では困難である。貨客混載での実証実験では積み替えの手間や輸送コストなどの問題が指摘されている[93]。なお、約40年の時を経て同様のコンセプトを持つ列車が在来線でJR貨物M250系電車（スーパーレールカーゴ）として登場した[94]。
東海道新幹線建設時における「貨物新幹線」計画

「貨物新幹線」は、東海道新幹線の建設時から東京-大阪間を5時間半で結ぶ夜行貨物列車の運行構想があった[95][注 11][96]。1958年（昭和33年）に国鉄幹線調査会が答申し、国鉄の新幹線総局計画審議室などが検討をおこなった[97]。

計画を担当していた石井幸孝によると、国鉄では5トンコンテナの規格を新幹線の車両限界に合わせたサイズ（8×8×12フィート）とし[97]、貨物取扱駅の用地買収を進め工事も開始したが、東海道新幹線の建設費がインフレの影響で当初の計画より二倍近くに膨れ上がったため計画を断念した[97]。その後、貨物新幹線用地は東京貨物ターミナル駅や車両基地などに転用された。大阪貨物ターミナル駅の近くには未完成の高架施設が残っていたが[97]、2013年から順次撤去工事が進められている[98]。

東海道新幹線建設時の計画については、世界銀行から新幹線建設の資金を調達する際、貨物が鉄道輸送の主力となっていたアメリカの理解を得るためのダミー構想であったとの見方があるが、石井はこれを否定している[97]。また、国鉄在籍時に東海道新幹線建設に従事した長尚は、高橋団吉著『新幹線を走らせた男 国鉄総裁十河信二物語』の「じつは、貨物新幹線を走らせる気は、最初からなかったのである。」という記述に対して「東海道新幹線の計画から開業後の暫くまで、国鉄の責任者・関係者は貨物輸送を真剣に考えていたことは、次のような幾つかの事実から間違いない」としている[99]。

    東海道新幹線の建設基準にある活荷重（列車荷重）は、N標準活荷重（貨物列車荷重）とP標準活荷重（旅客列車荷重）とからなっていて、平成14年（2002年）に改正されるまで、この基準は生きていた。
    貨物駅のための用地買収が各地でなされていた。
    貨物列車を引き込むための本線を跨ぐ施設が造られていた。しかも大阪鳥飼の車両基地への貨物引き込み用の構造物は世銀からの借款の調印の昭和36年5月2日から一年後に着工している。もし世銀から融資を受けるためであるならば、予算膨張に悩んでいたので、前項の用地買収など無駄なことはしなかったはずである。
    東海道新幹線開業後の昭和40年3月15日の国会の法務委員会で、国鉄常務理事が「国鉄といたしましては、新幹線を利用いたしまして高速の貨物輸送を行なうということが、国鉄の営業上どうしても必要なことでございまして、またその需要につきましても十分の採算を持っておりますので、なるべく早く新幹線による貨物輸送を行ないたい、こういうふうに考えて計画を進めております」と答弁している。

JTBキャンブックス『幻の国鉄車両』 pp. 46 – 52 には、コンテナ電車他各種車両のメトリクスや編成図が掲載されている。

石井は将来的には人口減少により旅客輸送の需要が減りダイヤに余裕が出来るため、最高速度を200km/h程度に抑えた夜行貨物列車が可能となると主張し、新型の導入で余剰となった旧型車両を改造してヤマト運輸のロールボックスパレットを搭載出来るようにした図面を公開している[100]。
山陽新幹線における検討

山陽新幹線建設の際に「路線の有効長は、将来の貨物運行を考慮して500mとしている。」[101]また、仁杉巌は「いずれ新幹線による貨物輸送をもとめる声も高くなってくるだろうと思いますが、そのときは、おそらく新幹線が関西よりももっとさきへのびたときではないかと考えられます。つまり、現在建設中の山陽新幹線が下関までのびたときとか、博多までのびたときとかです。」[102]と記している。なお、泉幸夫貨物局長は、博多開業前に「東海道新幹線を作りましたときに、将来は貨物もコンテナ輸送をやるという前提がありました。昭和34年に登場した5トンコンテナも、今縦積みにすれば、新幹線で使えるようにできているわけです。しかし、その後、100キロ貨車が開発され、東京 - 大阪間は、8時間で走るようになりましたから新幹線の貨物輸送は将来博多まで延びたときに、検討するという感じだったのです。その博多開業時期も大体きまってきたわけですが、貨物局を中心に勉強しまして、100キロ程度を出せるコキ車を使うと、博多から東京までといっても、そう時間に大きな差があるわけでもないことと、新幹線で5トンコンテナを運んでみたところでフリークェンシーに富んだ輸送は必ずしも期待できないこと等から、現時点では新幹線による貨物輸送は原則として考えていないのです。」[103]と述べている。
東北・上越新幹線における検討

東北・上越新幹線建設の際にも貨物輸送の可能性が検討されたが、迅速性を優先されるという物資の輸送では航空貨物との競合が避けられずしかも貨物シェア自体が小さいこともあって、基本規格は東海道・山陽新幹線に準じることとして将来の可能性を残す形で見送られた[注 12][104]。
レールゴーサービスの拡大案

1981年から、東海道新幹線で「レールゴーサービス」という小荷物などの運送がおこなわれていた。1982年には郵政省からの提案で、1985年頃には国鉄内から、これを拡大した郵便・貨物輸送が計画され、車両の設計図面まで存在していたが、いずれも実現しなかった[105]。
北海道新幹線における検討

2005年（平成17年）から建設が始まった北海道新幹線は、青函トンネルとその前後の区間を在来線の貨物列車と共用するため、同区間では片道あたり新幹線・貨物それぞれ2本/時しか走らせることができないと予想されている。JR北海道ではこのボトルネックを緩和する方法の一つとして、在来線の貨車をそのまま搭載する専用列車（トレイン・オン・トレイン）の研究を行っていた[106][107]が重量の問題から、E5系やH5系をベースにした貨物専用の新幹線車両の開発を検討している[108]。
JR旅客各社による貨客混載の事業化
「貨客混載」も参照

2020年以降、新型コロナウイルス感染症の社会・経済的影響で、新幹線の利用が大幅に減少し減収が続いているため、JR旅客各社では新幹線による荷物輸送の事業化に乗り出している[109][110]。これらは台車に乗せた荷物を使われていない座席と座席の間に置くなど、既存の客車をそのまま利用しているため大型の貨物には対応できず[109]、常温輸送となるため梱包に手間がかかり、青森から大宮までホタテを輸送した例ではトラックの4 - 5倍のコストがかかるなど不利な点も多い[93]。しかし高速ながら事故率の低さと定時性の高さがあるため、水産物や電子部品など特性にあった小荷物の高速輸送に使われている[111][93]。

JR東日本は、2017年から不定期で地域の特産品などを新幹線で運び駅ナカの見本市などで販売する取り組みを行ってきた[112]。2018年には、日本郵便と共同で、宮城県で収穫された農産物を東北新幹線で東京駅まで輸送して、東京駅でのイベントで販売したり[113]、2019年には、新潟県で水揚げされた海産物を上越新幹線で輸送し、品川駅の鮮魚店で販売したりする[114]試みや、2020年には、弘前市で収穫されたトウモロコシを朝に新青森駅で積み込み、当日に東京駅で販売するなど小口の高速輸送の手段[注 13]として検証が行われている[115][116]。2020年10月からは仙台駅 - 東京駅間の輸送を定期化した[112]。現在では駅ホームで積み込みが行われているが、作業スペースが狭く時間の制限もあるため、車両基地へ貨物を搬入し積み込みを行うなど効率化の試みもなされている[93]。

JR西日本も、JR東日本と連携して、北陸エリアの特産品などを北陸新幹線で首都圏に輸送する事業を拡大することを表明した[112]。JR西日本は、JR九州とも連携して、山陽・九州新幹線を活用して九州の特産品などを関西へ輸送する実証実験を2021年2月から開始する[112]。新大阪駅で荷物を降ろした後、在来線に積み替えて大阪や京都に輸送する貨物輸送の事業化を目指と発表した[112]。2022年11月4日には客車による即日配送を行う特産品ブランドを「FRESH WEST」と命名し[117]、19日から朝に鳥取県の境港市で水揚げされたズワイガニなどを米子駅から特急に乗せ、途中で新幹線に積み替えて新大阪駅へ輸送し駅ナカで販売する事業を開始した[111]。

JR北海道とJR九州も、それぞれ佐川急便と提携して宅配荷物を新幹線で輸送する事業を構想している[112]。JR北海道では、北海道・本州間（新函館北斗駅 - 新青森駅）で客室内に宅配荷物を収納した専用ボックスを積載して輸送する方式で、早ければ2020年度内にも正式に事業化する予定[112]。JR九州も、九州新幹線の博多駅 - 鹿児島中央駅間の上下で車内の余剰スペースに佐川急便の宅配便荷物を収納した専用ボックスを積載して輸送する貨客混載事業を2021年5月18日から開始し[118]、また旧車販準備室を使用する荷物輸送サービス「はやっ!便」を同日に開始した[119]。
国、JR貨物とJR旅客各社による貨物新幹線の検討

2021年1月8日、日本貨物鉄道（JR貨物）は、現在進行中の「JR貨物グループ中期経営計画2023」に続く長期計画として「JR貨物グループ長期ビジョン2030」を発表した[120][112]。JR旅客各社と連携して「貨物新幹線」の検討を進め、期間中に実現したい考えを示した[120][112]。「物流イノベーションや既存鉄道インフラの有効活用（人流・物流の一体化による鉄道事業の持続性向上）として貨物新幹線の検討を推進」という文言で、貨物新幹線の事業化に言及している[121]。

2022年、国土交通省は、深刻なトラックドライバー不足（2024年問題）や2050年までのカーボンニュートラル実現に向けた対応の必要性など、物流における諸課題の解決を図るため、「今後の鉄道物流のあり方に関する検討会」を設置した[122]。7月28日に開かれた第5回の会合では、鉄道貨物の輸送量拡大やJR貨物の経営自立支援に向けた提言の中間とりまとめを行った[123]。提言では「貨物鉄道の輸送モードとしての競争力強化」「貨物鉄道と他モードとの連携」「社会・荷主の意識改革」の3つの視点から14の課題を抽出した[123]。貨物鉄道は、航空機やトラックと比べて、ドアツードアではリードタイムや輸送品質の面で十分に競争力がある輸送サービスを提供できていないが、新幹線の貨物輸送の拡大によって競争力を強化することを課題の1つに挙げている[123]。新幹線は旅客輸送に特化する前提で整備されてきたが、近年は一部で貨客混載輸送が行われ、宅配便、鮮魚・鮮果、半導体といった付加価値の高い品目の輸送において、新たな市場を開拓しつつある[111][124]。将来的に、新幹線の貨物専用車両による高頻度の大量高速輸送を実現できれば、日本の物流においてイノベーションを引き起こす可能性があり、新たな輸送ニーズや市場の開拓を通じた日本経済・地域経済の成長への貢献、航空機やトラックからのモーダルシフトによる地球環境への貢献の可能性も出てくる[124]。一方で、大量輸送と高速走行を兼ね備えた車両の新規開発、積み替え技術の開発と必要な施設の整備、乗入れ区間、旅客列車とのダイヤ調整、安全確保のための方策、運行主体や費用負担など、多くの課題があることが指摘されている[124]。国、JR貨物、JR旅客会社等の関係者による調査を行って課題を整理した上で、まずはJR貨物が中心となって、線路容量に余裕がある路線における走行を念頭に置いた、高速走行と大量輸送の両立が可能な貨物専用車両の導入の可能性を検討する必要があるとしている[124]。
運賃・特急料金
運賃

新幹線の運賃は、並行在来線の営業キロを元に決められる。これは元来新幹線が並行在来線の別線増設として建設されたという歴史的経緯や、運賃計算の繁雑化を避けたことによるものである。詳しくは以下の通り。

注:この節での（運賃・料金計算のための）「並行在来線」とは、東海道新幹線では東海道本線、山陽新幹線では東海道本線・山陽本線・鹿児島本線、東北新幹線の東京駅 - 盛岡駅間では東北本線、上越新幹線では（東北本線）・高崎線・上越線・信越本線、九州新幹線の博多駅 - 新八代駅間、川内駅 - 鹿児島中央駅間では鹿児島本線、西九州新幹線の諫早駅 - 長崎線間では長崎本線のこと。

    新幹線と並行在来線は原則として同一路線とみなされる（「幹在同一視」）。そのため、新幹線を利用した場合と在来線を利用した場合とで基本的に運賃は変わらない（後述するように例外もある）ので、在来線用の乗車券に特急券を買い足すことで、新幹線を利用できるようになる。
        山陽本線には岩国駅 - 櫛ケ浜駅間を含む区間について岩徳線経由のキロ数で運賃を計算する特例がある（経路特定区間）が、山陽新幹線にもこの特例が適用される。なお、山陽新幹線の実際のルートは岩徳線沿いになっている。
    並行在来線と接しない新幹線駅については、それに最も近い（もしくは対応する）並行在来線の駅に相当するものとして営業キロを定める（例:新花巻駅は花巻駅の営業キロを用いる）。
    並行在来線（の一部）が廃止されたり第三セクター鉄道に転換されたりして「並行するJR線」が消滅した区間（北陸新幹線の高崎駅 - 敦賀駅間・東北新幹線の盛岡駅 - 新青森駅間・九州新幹線の新八代駅 - 川内駅間）については、実際のキロ数を営業キロとする。
    幹在同一視の原則により、片道乗車券の経路に新幹線とそれに対応する区間の並行在来線の両方を含むことはできない。
        例として名古屋→（新幹線）→静岡→焼津と乗車する場合、静岡 - 焼津間が重複となるため1枚の片道乗車券にはできず、名古屋→静岡と静岡→焼津の別々の乗車券が必要である（連続乗車券にすることもできる）。

一方、新幹線と並行在来線とを完全に同一視すると旅客にとって不利になる場合を考慮して、以下のような例外がある。

    並行在来線と接しない新幹線駅を含む区間（例：品川 - 新横浜 - 小田原）については別の路線として扱う。ただし、乗車券の発駅・着駅・他線との接続駅のいずれかが当該区間内（両端を除く）の駅である場合に限る。
        例えば大阪→新大阪→（新幹線）→名古屋→大垣と乗車する場合は、米原 - 名古屋間に並行在来線と接しない新幹線駅（岐阜羽島）があり、かつ乗車券の着駅（大垣）がこの区間内のため、名古屋 - 大垣間を重複とせず全体を1枚の片道乗車券にすることができる。一方、大阪→（新幹線）→名古屋→（在来線）→米原と乗車する場合は、条件を満たさないため1枚の片道乗車券とはならない。

また山陽新幹線の新下関駅 - 小倉駅 - 博多駅間については、新幹線（JR西日本）と在来線（JR九州）とで管理する会社が異なることから、他の区間とは扱いが異なっている。

    基本的には同一の路線として扱うにもかかわらず、運賃が異なる。
        JR九州管内となる下関以西の在来線では乗車距離に応じた加算額が課されるのに対し、JR西日本管轄の新幹線ではそれがないため。
    運賃が異なることに起因して、片道乗車券の発売条件の判定がかなり煩雑である。規則を厳密に解釈すると、条件によっては片道乗車券でも連続乗車券でも発売できない経路が存在する。

詳しくは、旅客営業規則第16条の2、第16条の3および第16条の4を参照。

    JRグループ旅客営業規則 - JR東日本

特急料金

新幹線（山形・秋田新幹線を除く）の特急料金は、乗車券や在来線の特急列車のような対キロ制ではなく、各駅の区間ごとに決められた、いわゆる三角表方式となっている。

新幹線と在来線の乗り継ぎについては、一定の条件で在来線の特急・急行料金を半額に割り引く制度がある（乗継割引）。これは、新幹線が開業する前は1本の（特急等の）列車で済んでいたものが、開業したことによって複数の（新幹線と特急等の）列車に分割されることによる合計後の特急料金等の負担増を軽減することをそもそもの目的として設けられたものである。

なお山形新幹線と秋田新幹線については、新在直通運転を行うため特殊な特急料金を設定している（「山形新幹線#運賃と特急料金」および「秋田新幹線#運賃と特急料金」を参照）。
営業上の競合など
	
この節には独自研究が含まれているおそれがあります。問題箇所を検証し出典を追加して、記事の改善にご協力ください。議論はノートを参照してください。（2011年10月）
飛行機との競合

長距離輸送においては、従来から国内線との競合が続いていたが、航空会社の規制緩和による各種割引運賃の導入（早割、特割、激割など）や旅行業者と連携しての宿泊料金込みの格安プランの販売、および格安航空会社の参入等により、競争は一層激化している。

また、航空会社によるマイレージサービスの存在も大きく影響している。これは高頻度の利用客に対し通常より多いボーナスマイルや専用ラウンジの用意、渡航先宿泊の割引などの高いサービスを与えて優遇する制度であり、利用者の大きな支持を得ている。鉄道側でも、例えばエクスプレスカードの場合、会員に対しポイントシステムを開始しているが、そのサービス内容や、高頻度利用客への優遇サービスは格段に異なっている。

航空会社との対抗については、航空路線と競合する区間を中心に割引率の大きい特別企画乗車券の発売や、ビジネス客の多い東海道・山陽新幹線ではJR東海エクスプレス・カードとJ-WESTカード（エクスプレス）による「エクスプレス予約」、東北・山形・秋田・北海道・上越・北陸新幹線では「えきねっと」といった、運行会社自身の会員制インターネット予約による割引特急券の発売が行われている。とりわけ2006年（平成18年）の神戸空港や北九州空港の開港は、競合する東海道・山陽新幹線への影響が大きく、「エクスプレス予約」の山陽新幹線への拡大、N700系車両の共同開発など、それまで対立の多かったJR東海とJR西日本両社は連携を強化する体制に転換しつつある。一方、航空会社も東京 - 大阪間でのみ使える予約変更自由、航空会社選択自由のシャトル便往復割引を導入して迎え撃っているほか、羽田空港の滑走路増設による発着能力増強や、横田空域の一部返還により、更なる所要時間短縮による競争力強化が見込まれている。また、京浜急行電鉄や名古屋鉄道といった空港連絡鉄道路線を持つ鉄道各社との連携も行っている。

山陽新幹線においては、終点である博多駅と福岡空港がほぼ隣接している（地下鉄で2駅、5分前後）という特徴もあり、福岡 - 名古屋間では新幹線と航空会社との競争が非常に激化している。福岡 - 大阪間は従来競争が激しかったが、「ひかりレールスター」の登場などにより、鉄道側が優位に立っている。さらに2011年の山陽新幹線と九州新幹線の直通運転開始により、従来は航空側が優位であった大阪 - 熊本・鹿児島間でも、新幹線と航空会社との競争が激化し始めている。

また、2015年の北陸新幹線金沢延伸後は東京 - 金沢間でも新幹線と航空会社（小松空港発着）との競合が激化し始めている。

このように競合している一方で、新幹線が災害や事故などで運転を見合わせた場合などには、航空路は新幹線の代替交通機関としての機能も果たしている。
他の鉄道との競合
私鉄特急との競合

東海道新幹線の開業以来、新幹線と競合した私鉄特急としては、近畿日本鉄道（近鉄）、小田急電鉄、名古屋鉄道（名鉄）の特急があった。

私鉄特急はいずれの場合も、所要時間では新幹線と比較して大きく不利なので、割安な運賃・料金、駅の立地、車内の居住性などで対抗することになった。

近鉄特急との競合
    直接の競合は、名古屋 - 大阪間で見られる。大阪側では、新大阪駅から離れたミナミに対して、乗換を必要としないエリア（難波、鶴橋など）があることなどから、近鉄特急にも優位性がある。
    競合は1964年（昭和39年）の東海道新幹線開業時に始まる。当初は運賃・料金でも差が小さかったことや、所要時間の大幅な差などから利用客を新幹線に次々と奪われた。大阪万博のあった1970年（昭和45年）を除き、1970年代前半までは低迷が続き、近鉄の名阪ノンストップ特急（甲特急）は汎用車両の2両編成による運行を余儀なくされた。一時は単行車両の導入も検討されたといわれている[注 14]。
    しかし、1970年代後半以降は、国鉄の頻発する運賃・料金の値上げとストライキに対する嫌気から、名古屋 - 大阪間においては、特に急がない個人客を中心に、新幹線から近鉄特急への乗客移行が多く見られた。その結果、1980年代に入ると同区間の近鉄特急も3両編成、後には6両編成にまで復調したが、運用される車両は汎用車両のままであった。
    その後、100系車両の投入（1985年）とJR東海の発足（1987年）による東海道新幹線の競争力強化を受けて、1988年（昭和63年）に近鉄特急も新型車両「アーバンライナー」を投入し、2000年代には更なる新型車両「アーバンライナーnext」投入や「アーバンライナー」の「アーバンライナーplus」へのリニューアルを実施、運賃面でも割引乗車券の名阪まる得きっぷを導入するなどして、主に運賃面と快適性をアピールした。
    なお、伊勢志摩・奈良方面など新幹線と競合しない区間では、むしろ東海道新幹線と近鉄特急は補完関係となっている。1964年（昭和39年）の東海道新幹線開業で、近鉄は自社特急網を新幹線の培養ルートとして育成し、新幹線で大阪・京都・名古屋に到達した旅客を自社沿線の観光地へ誘致する戦略を採った。伊勢志摩方面では在来線列車「みえ」との競合も多少見られるが、JR東海の特別企画乗車券の中には、新幹線と接続する京都駅から奈良方面への移動にJR西日本の奈良線ではなく、近鉄線を指定しているものも存在する。
    なお、上述した歴史的経緯の詳細は、近鉄特急史に詳しい。
小田急特急との競合
    東京 - 小田原において、「はこね」などとの競合が見られる。ただ、運賃格差の大きさと箱根方面への輸送を含むというその性質の差、それに東京側ターミナルの違い（新幹線・東京駅、小田急・新宿駅[注 15]）などが作用して、棲み分けがされている。
    東海道新幹線を運営するJR東海とは、国鉄時代から継続して小田急から御殿場線に乗り入れて新宿 - 御殿場間を運行する「ふじさん」（2018年までは「あさぎり」。一時は運行区間を新宿 - 沼津間に拡大し、専用車両として371系・20000形RSEを開発して共同運行を行っていた）を設定するなどしており、対立関係は見られない。
名鉄特急との競合
    愛知県の名古屋 - 豊橋について競合が見られるが、JR東海は並行する東海道本線において新快速などを運行しており、むしろこちらが名鉄特急との直接の競合関係にある。なお、この区間の新幹線利用を促進するために、在来線用の「名古屋-豊橋カルテットきっぷ」と併せて使うと新幹線に乗れる「カルテットきっぷ専用新幹線変更券」が販売されている。

JR（国鉄）同士 の競合

国鉄時代、新幹線に並行する在来線特急と競合したものの、同一事業者よる運用として一般的には「競合」とはみなされていなかった。しかしながら、国鉄民営化後は新幹線と在来線特急が別会社によって運用されるケースが発生し、営業的にも競合関係となった。具体的な例は以下の通りである。

福岡県内（北部九州）
    山陽新幹線の小倉 - 博多間がそのひとつである。JR西日本（山陽新幹線）は博多駅発着の「のぞみ」などを増発し、朝晩の通勤時間帯を中心に小倉 - 博多間のみの「こだま」も設定している（詳細は山陽新幹線#福岡県内を参照）。また、土休日には新幹線よかよかきっぷを販売しており、在来線特急の2枚きっぷと大差ない運賃で利用できる。一方、JR九州（鹿児島本線）では特急「ソニック」「きらめき」の増発、特急料金の値下げや885系・883系・787系の投入などで対抗していたが、2010年代に入り日中の「きらめき」の廃止や在来線普通・快速と大差ない運賃で利用できる「10枚きっぷ」の廃止などが行われ、新幹線に対する優位性が低下している。2018年には鹿児島本線の快速の減便（博多駅近郊のみの区間快速化）も行われており、小倉 - 博多通しの需要ではなく、新幹線の利用が不便な途中駅からの需要を担う路線にシフトしつつある。なお、鹿児島本線の小倉 - 博多間は輸送密度（平均通過人員）がJR九州内で最大の区間となっている（2016年度）[125]。そのほか、この区間には西鉄バスによる低廉（片道1,130円）な予約不要の高速都市間バス「ひきの号」「なかたに号」「いとうづ号」も運行されているが、こちらも新幹線との競合により減便に追い込まれている。
南関東（神奈川・伊豆方面）
    東京・品川 - 熱海において、JR東日本・東海道線との競合が見られる。平日早朝に上り、夜間に下り特急「湘南」、を運行し、特に東京発 18 - 20時台では特急「湘南」が 00分・30分発と2本体制になっている。ただ、平日かつ早朝・夜間帯のみの運行であるため、競合の主な対象が通勤者に置かれており、日中の時間帯や休日では棲み分けがされている。また、快速・普通列車のほぼ全ての列車には2階建てグリーン車も連結されていて、小田原・熱海方面からの在来線利用の遠距離通勤者に配慮している。
    他には、東京方面からの富士箱根伊豆国立公園方面へのアクセスにおける、東京・品川 - 熱海間（JR東海・東海道新幹線とJR東日本・東海道本線）の競合があげられる。両社はこの区間において在来線同士の直通運転を除き、新幹線と在来線の相互連携は特に見られず（国鉄時代から発売されている特別企画乗車券「伊豆フリーQきっぷ」で、東京 - 熱海 - 三島間で東海道新幹線あるいは在来線特急（踊り子号）自由席の利用が可能である程度）、JR東日本側では在来線特急を伊豆急行線や伊豆箱根鉄道駿豆線と東京を直通させている。
    なおJR東海は熱海駅、小田原駅から東京都区内までは新幹線を利用し、東京都区内JR在来線・りんかい線が乗り降り自由となる「こだま都区内・りんかいフリーきっぷ」を2013年7月31日まで発売していた[126]。
近畿圏
    米原 - 新大阪・大阪間についてJR東海が運行する東海道新幹線とJR西日本が琵琶湖線・JR京都線にて設定している新快速で競合しているが、新幹線の停車駅が米原・京都・新大阪しかなく、新幹線の利用が不便な地域が多いため競合範囲は限られている。新幹線は長距離輸送が主体で、都市圏輸送を担う新快速と棲み分けがなされている。
青森市 - 今別町
    奥津軽いまべつ駅と津軽二股駅が隣接するJR北海道の北海道新幹線とJR東日本の津軽線については青森市側の起点が新青森駅と青森駅で異なることもあり、棲み分けられている。また、この区間は並行在来線ではないという公式見解があるほか、出発または到着駅を同一駅とした場合でも選択乗車はできない。

高速バスとの競合

高速バスは「定時性・速度では劣るが、時間・経路・発着地の柔軟性と価格で優れている」という性質がある。 そのため、長距離区間では不利となるが、中距離区間や、大都市間を初めとする区間では、夜行バスで格安で移動できることから、高速道路の整備が進んだ1980年代ごろから人気となっている。また、都心部（東京の新宿や渋谷、名古屋の栄、大阪の梅田や難波、広島の紙屋町、福岡の天神など）やテーマパーク（TDRやUSJなど）に直接乗り入れており、新幹線駅間以外でも競合している。 しかし、上述のように性質が大きく異なるため、「速さを取るか、安さを取るか」という直接的な競合というよりは、発着地、所要時間といった利用者のニーズで使い分けられている。例えば、目的地が遠く、早朝に到着する必要がある場合は、より早い時間に到着する高速バスが有利となる。早朝から（あるいは夜間の）在来線特急との乗り継ぎを要する場合はさらに高速バスが有利となる[注 16]。

その一方、在来線特急との乗り継ぎが現実的でない地域では、新幹線駅との接続に高速バスが用いられる場合もあり（B&Sみやざき等）、高速バスが競合ではなく補完の役割を担う場合もある。
その他
政治の影響
詳細は「鉄道と政治#新幹線と政治」を参照

新幹線の建設に関しては、その開業効果が大きいことから、沿線の利害に関係することとして建設時よりさまざまな政治介入がなされてきたといわれる。

最も古い話では、東海道新幹線の建設時に起こった京都駅の設置是非をめぐる問題や、大野伴睦の介入による岐阜羽島駅の設置騒動がある。ただし岐阜羽島駅の設置には、関ヶ原の降雪対策という政治的な影響力とは別の理由もあり、政治力のみで設置されたわけではないと言われている。

また逆に、一度は着工された駅新設が、その新設を争点とした選挙での県知事交代によって凍結に追い込まれた、滋賀県の南びわ湖駅の例もある。
世界の高速鉄道の呼称

日本では、新幹線という単語がすでに高速鉄道そのものを意味する普通名詞と化しているため、報道などでは日本国外の高速鉄道についても国名を付けて「○○新幹線」「○○版新幹線」「○○の新幹線」と広く呼ばれている（例：TGVはフランス新幹線、ICEはドイツ新幹線、KTXは韓国新幹線、ロシア高速鉄道はロシア新幹線、台湾高速鉄道は台湾新幹線など）。

しかし、日本の新幹線は車両・軌道・架線・信号 (ATC) などを総合した独自のシステムであり、ミニ新幹線を除けばヨーロッパのように在来線と相互乗り入れしているわけではなく、他の高速鉄道システムとは区別することがある。英語では、日本の新幹線は Shinkansen と表記されるように、新幹線とは日本の高速鉄道システムの固有の名称として取り扱っている。技術的には、他国の高速鉄道と異なり在来線とは独立したシステムとなっているのが特徴で、動力分散方式など独自性が強いのも特徴である。もっとも、車両が新幹線とほぼ同一であるなど仕様が近い台湾高速鉄道に関しては、現地でも高雄捷運の日本語アナウンスなどで「台湾新幹線」と呼称している例も存在し、同鉄道の建設の際に機電システムを請け負った日本連合7社が設立した合弁会社は「台湾新幹線株式会社」と名乗っていた[127]。
夜行新幹線計画

山陽新幹線開通前に「夜行新幹線」も計画され、山陽新幹線技術基準調査委員会報告（1966年）では、東京から博多の間を一晩に計24本で運行した場合、片道平均5,000から7,000人の需要があると見込んでいた。新幹線の夜間運行は片側1路線を運用し、もう片側の路線は保守点検して運行するという計画であった。また当時は四国新幹線や中国横断新幹線の計画も含め衝突を避けて夜間運行を実現させるために姫路駅の新幹線13番ホームを待避線に、待避駅として西明石駅・相生駅が建設された[128]。しかし名古屋新幹線訴訟などの新幹線の騒音問題が浮き彫りになったことや国鉄分割民営化で夜行新幹線の計画は実現しなかった。

大規模なイベントにあわせて、臨時列車として夜行新幹線が運転されたことはある。2002 FIFAワールドカップの際には東海道新幹線・上越新幹線で深夜帯に臨時列車が運転された[129]。2020年東京オリンピックでは、宮城スタジアムで行われるサッカー競技の試合に合わせ、東北新幹線の仙台駅 - 盛岡駅間、仙台駅 - 東京駅間で臨時の夜行新幹線を走らせる予定があったが[130]、新型コロナウイルス感染症の流行拡大に伴い、宮城では有観客の開催となったものの、感染拡大防止の観点から夜行新幹線の運転は中止され幻となった[131]。

大阪産業大学工学部の波床正敏・井上喜裕らのように、新幹線の夜行運行の適用可能性を環境負荷と発着時間帯の観点から検討し、発着時間帯の設定自由度が従来の夜行列車より高く有望であるとする考えもある[132]。
駅での新幹線案内サイン・駅名標

    新幹線が乗り入れる駅において、駅構内の表示では、ピクトグラムとして国鉄時代は0系・200系を元にした絵（丸型の先頭車両）が描かれていた。JR東日本の東北・上越新幹線の駅、および東京駅での東海道新幹線乗り場案内サイン（JR東日本構内）は現在もこれを踏襲している。しかし、JR東海と西日本では、その後登場した車両の絵を用いている。
    地下鉄など国鉄・JR以外の駅では、乗り換え表示に「JR線」と表示するのではなく「JR線・新幹線」と新幹線を在来線と分けて記載する例がみられる。
    新幹線の英語表記の案内表示では、表記が統一されていないものがある。たとえば、「新横浜」をShin-Yokohamaと表記しているところもあれば、Shin-yokohamaと表記しているところもある。この点は専門家の間でもまとまった意見は出ていないのが現状である。
    開業当初より1970年代中頃までの東海道新幹線では、在来線と異なり同線の独自仕様の駅名標を採用した。様式としては在来線の駅名標と比較して横長となり、また漢字と全大文字のローマ字表記のみとして、平仮名表記を省略し、さらに在来線の駅では下部に書かれている前駅と次駅の表示も廃止した[注 17]。同じく開業当初の山陽新幹線でも独自仕様の駅名標が採用されたが、こちらは東海道新幹線とは異なり、漢字と全大文字のローマ字表記の自駅表示に前駅と次駅の表示を追加したものであった。しかし東北・上越新幹線以降の新幹線では新幹線独自の駅名標は採用されず、東海道新幹線では1970年代後半頃から、山陽新幹線では国鉄末期からそれぞれ急速に在来線や東北・上越新幹線以降で採用された国鉄標準の様式の駅名標に順次交換され、JR発足当初にはこの駅名標は既に現存しなくなっていた。なお、JR化後はJR各社が独自の様式の駅名標を採用し、全駅それに取り換えられているため、2015年現在では新幹線では国鉄型の駅名標は皆無となっている。

警笛・走行音など

    新幹線の「音」は「ビュワーン」という擬音表現が古くからよく知られ、メディアなどでも取り上げられることが多かった。新幹線を用いた旅行という設定のCM、童謡『はしれちょうとっきゅう』（作曲：湯浅譲二、歌・作詞：山中恒）の歌詞などである。これを新幹線の走行音と思う人も多かったが、実際は走行中の主に高速走行時の警笛音である。なお、この音で新幹線がイメージされることは1980年代以降薄れ、100系以降の東海道・山陽新幹線車両や東海道・山陽新幹線以外の新幹線についても、この音がメディアなどで取り上げられることは少なくなった。
    実際の新幹線の走行音は、低速運転時（少なくとも110 km/h以下）の場合は在来線の走行音より静かである。これはロングレールの多用によりレールの継ぎ目が少ないことや、在来線車両よりも歯数比が低く、同じ速度であれば電動機がより低速で回転することなどに起因している[注 18]。
    走行音の発生源としては車輪や架線、車体前面や側面・上面の突起物による風切り音（空力音）がある[注 19]が、300 km/h近くなると空力音がその大半を占めるようになる。そのため高速走行には「新幹線車両」で述べたような空力音対策が必要とされる。
    トンネル突入の際、圧縮された空気により、退出側の坑口周辺ですさまじい騒音が発生する。トンネル微気圧波による騒音であり、圧縮波とも呼ばれている（通称「トンネルドン」）。上記に同じく、対策が必要とされる。

列車ダイヤ

新幹線については、異常時を除き午前0時から午前6時までの列車の運転を営業・非営業列車を問わず実施しておらず[注 20]、保線作業のための時間に充てられている。このため、始発列車に使われる車両は、前日の深夜に整備済みの編成を車両基地から駅に回送し、そのまま夜間滞泊させる場合が多い。ただし、北海道新幹線の青函トンネル界隈の区間では、在来線と共用しており、在来線の貨物列車は午前0時から午前6時までの間にも運転されている。また、在来線に直通する山形新幹線及び秋田新幹線の在来線区間（福島駅 - 新庄駅間、盛岡駅 - 秋田駅間）では運行時間帯の制約がなく、この時間帯にも運転される列車がある。
営業主体の根拠

法律面では新幹線の営業主体を特定していないが、運営がJRグループに継承されている理由としては、

    新幹線の経営には莫大な費用がかかり、それを負担できる資本力があるのは旧国鉄の業務を継承したJR各社しか存在しない。
    旧国鉄には、東海道新幹線、山陽新幹線、東北新幹線、上越新幹線を経営してきた実績があり、それがJR各社に分割民営化されたことで、運営を知る人材を持つJR各社に引き継がれた。

ということが挙げられる。
地名における「新幹線」

静岡県田方郡函南町には「新幹線」という地名が存在する。これは昭和30年代の新幹線計画にちなむものではなく、戦前の弾丸列車計画時代のものである。弾丸列車計画時代に新丹那トンネルの工事を行うための従業員宿舎が置かれた場所で、工事終了後に宿舎は撤去されたが、その後同地に住宅団地が建てられ「新幹線」という地区が生まれることとなった。この地区には新幹線公民館や「幹線下」という名のバス停も存在している。

東京都国分寺市の鉄道総合技術研究所のある場所の地名は「光町」である。国分寺市が1966年に町名整理を行った際、同研究所での新幹線開発と1964年の東海道新幹線開業を記念し、列車愛称「ひかり」にちなんで旧地名の平兵衛新田から改称したものである。由緒ある旧地名のため研究所は地元市民との交流の機会にもなっている一般公開を「平兵衛まつり」と名付けている。
送電線名における「新幹線」

鉄道路線ではなく、送電線の名称に「新幹線」と付けられたものがある。例として、猪苗代新幹線・飛騨新幹線等があり、いずれも東海道新幹線が開通する1964年（昭和39年）よりはるか以前の大正末期から昭和初期に開通しており、「新幹線」の語を最初に使ったのは旧国鉄ではなく電力会社である。
イベント列車
0系新幹線を模した「鉄道ホビートレイン」

新幹線が走っていない四国旅客鉄道（JR四国）では、2014年より予土線でキハ32形気動車を改造した“新幹線風の”車両「鉄道ホビートレイン」が走っている。非電化区間を走行するために、モーターではなく、ディーゼルエンジンを動力にしている。車内には0系風の転換座席を2脚設置したほか鉄道模型も展示されている。また、汽笛は新幹線のものと同じ音が鳴る。
新幹線を題材にした曲
[icon]	
この節の加筆が望まれています。
「東海道新幹線#開業記念歌」も参照

    『はしれちょうとっきゅう』（作曲：湯浅譲二、歌・作詞：山中恒）
    『ヤッホー!しんかんせん』（作詞：伊藤アキラ、作曲：SHIKAMON、歌：おがわみと） - フジテレビ「ひらけ!ポンキッキ」
    『新幹線でゴー!ゴ・ゴー!』（作詞：青島利幸、作曲：赤坂東児、歌：横山だいすけ・三谷たくみ） - NHK教育「おかあさんといっしょ」2011年6月の今月の歌
    『GO SHINKANSEN!』（CHERRY & LUKE） - 日本では「SUPER EUROBEAT VOL.202」に収録されている。
    『ひかり』（作詞：百田夏菜子（ももいろクローバーZ）、作曲：岡田実音・百田夏菜子、歌：百田夏菜子） - 東海道新幹線に乗車中の心情を表現しており、歌詞の中に「ひかり」「のぞみ」「こだま」が登場する[134]
    『岡山恋唄(新幹線は岡山まで)』（作詞：千家和也、作曲：田辺信一、歌:ありま双兵）

キャンペーン

東北新幹線の八戸駅 - 新青森駅間開通（2010年12月4日）、および九州新幹線の博多駅 - 新八代駅間開通（2011年3月12日）に合わせて、北海道旅客鉄道（JR北海道）・四国旅客鉄道（JR四国）も含むJRグループ旅客6社共同企画として、2010年12月15日より2011年3月31日まで『THE 新幹線』キャンペーンが行われた。キャッチフレーズは「日本とともに、走り続ける夢がある」。期間中は、新幹線の駅などをチェックポイントとし、携帯電話のGPS機能を活用したモバイルラリー「“THE 新幹線”ポイントラリー『日本列島縦断 チャレンジ新幹線！』」や、JRグループの鉄道・旅行情報サイト「トレたび」でのスペシャルサイトの開設などが行われた。
脚注
[脚注の使い方]
注釈

    ^ ただし山形新幹線の施設の一部は山形ジェイアール直行特急保有が所有していた。
    ^ a b 1997年（平成9年）10月1日の開業当初は東京駅 - 長野駅間の開業であり北陸には到達していなかったため、駅や時刻表などでの案内上は「長野行新幹線」、のちに「長野新幹線」の呼称が用いられた。2015年（平成27年）3月14日に金沢駅まで延伸されたため、案内上も正式名称の「北陸新幹線」となった。
    ^ 軌道回路には1000Hzの高周波を搬送波として流している、地上装置は開業当初のものである。
    ^ 三相交流式の電源系統から単純に2線を引き出して単相の大電力を利用すると、三相交流側に不平衡な電圧が生じ逆相電流によって過熱などの障害が発生する危険性がある。
    ^ 『新宿駅線群の下を抜く　地下鉄10号線シールド』「トンネルと地下」1978年3月号には都営新宿線と京王新線の建設にあたり、「国鉄横断部分（延長125ｍ）については、新幹線新宿乗り入れなどの将来構想を考慮して32/1,000の急勾配で下り、貨物線の下に至る」と記載されている。また、1976年（昭和51年）12月に行われた国鉄の第27回停車場技術講演会で発表された『新宿駅将来計画』には山手貨物線下の地下3階に3面6線の平面図が掲載されている。
    ^ 一部の「みどり」「ハウステンボス」も「リレーかもめ」として運行される。
    ^ 平日朝運転の高崎発東京行き「たにがわ472号」が本庄早稲田駅通過[37]。
    ^ 新京阪鉄道は、最終的な目標として名古屋までの延伸も視野に入れて建設され、名阪間を最高速度120 km/h、所要時間2時間で結ぶ構想も持っていたが、疑獄事件や不況などの影響もあって名古屋延伸構想は挫折した（名古屋急行電鉄の記事も参照）。
    ^ 過去、実際に東海道新幹線の車内において、1993年には覚醒剤中毒者が刃物で別の乗客を殺害した事件が、2015年には乗客が車内に持ち込んだガソリンによって焼身自殺を図るという東海道新幹線火災事件が、2018年には鉈を持った乗客が別の乗客ら3人を切りつけ、1人が死亡する事件が、それぞれ生じている。他にも、1967年には高校生によって「ひかり21号」の一等車座席下に爆弾が仕掛けられる事件が起きたが、この時は爆弾は車掌によって発見されたため事件は未遂に終わっている。
    ^ 前年に西ドイツ国鉄がアウクスブルク - ミュンヘン間で200 km/h運転を行ったが、短期間で中止しており、長期間にわたって行なわれたものとしては「ル・キャピトール」がヨーロッパ初である。
    ^ 現在九州新幹線に使用されている800番台の形式番号は、元々は貨物用新幹線車両の形式に予定されていたものである。
    ^ 雪害対策等で軸重が増加したことから、貨物輸送の有無での建設費用の差を考慮する程ではなくなったこともある。
    ^ かつて農作物を消費地へ迅速に輸送する手段として、農道離着陸場を利用した小口の航空輸送事業が行われたが採算が合わずに終了している。
    ^ ただし、1974年 - 1982年に行われた東海道新幹線の総点検および若返り工事に伴う半日運休日の午前中や、1975年のスト権ストで国鉄が全面運休に追い込まれた日には、名阪甲特急が4 - 6両に増結されて運転されたこともあった。
    ^ ただし、東京メトロ千代田線に直通する特急列車は東京駅に隣接する大手町駅に停車する。
    ^ この性質を指摘した例…BizStyle 〜ビジネスマンの高速バス活用術〜：琴平バスの比較広告。「夜間に移動するため、現地で早朝から遅い時間まで行動できる」と言う点で新幹線や飛行機に対する優位性が示されている。
    ^ この様式の駅名標は、東海道新幹線50周年記念で発行された各書籍の開業当時の駅構内の写真や1960年代中頃から1970年代中頃にかけて発行されていた鉄道雑誌の一部の頁で確認することが可能である。
    ^ 特に埼京線では、平行する東北新幹線よりも騒音が大きいと苦情が多数寄せられ、開業からわずか5年で新型車両(205系)に入れ替えなければならない事態となった。なお、新幹線側は当初から走っている200系が20年以上経った2013年まで同区間を走行していた。
    ^ 初期の0系時代にはパンタグラフの摺動音と電気火花による音もした。
    ^ ただし、車両基地と最寄り駅間の短距離区間については、この前後の時間に運行されることがある。例えば、2011年3月12日改正（九州新幹線全線開業）時の東海道・山陽・九州新幹線では、5時42分に熊本総合車両所を発車する回送が最も早く、0時18分に博多総合車両所及び熊本総合車両所に到着する回送が最も遅くなる[133]。

出典

    ^ 全国新幹線鉄道整備法（昭和45年法律第71号）第2条 - e-Gov法令検索
    ^ “翼なしで飛ぶような速さ、世界最速の列車10選”. CNN.co.jp (2022年1月31日). 2022年9月7日閲覧。
    ^ 国土交通省総合政策局情報政策課交通経済統計調査室、2016、 (Excel) 『鉄道輸送統計年報』28号（平成27年度分）、国土交通省
    ^ 全国新幹線鉄道整備法（昭和45年法律第71号）第4条 - e-Gov法令検索
    ^ a b 『読む・知る・愉しむ 新幹線がわかる事典』、p.19 - p.20。
    ^ a b 『読む・知る・愉しむ 新幹線がわかる事典』、p.53。
    ^ 『読む・知る・愉しむ 新幹線がわかる事典』、p.65。
    ^ 『読む・知る・愉しむ 新幹線がわかる事典』、p.67。
    ^ 全国新幹線鉄道整備法（昭和45年5月18日法律第71号）第2条 - 総務省 e-Gov検索
    ^ a b 『読む・知る・愉しむ 新幹線がわかる事典』、p.25 - p.26。
    ^ 東北新幹線工事誌－上野・大宮間－
    ^ 中央新幹線の整備計画の決定について (PDF) - 国土交通省、2011年5月26日。
    ^ a b 草町義和 (2018年4月18日). “「新幹線」大正時代からあった？　意味が変わった言葉の歴史”. 乗りものニュース. メディア・ヴァーグ. 2022年9月13日閲覧。
    ^ 野田隆 (2016年7月21日). “外国人に通じる？列車内や駅の英語表現10選 ｢山手線内回り｣は英語でなんと言えばいい？（2/4ページ）”. 東洋経済オンライン. 2022年9月18日閲覧。
    ^ a b 『読む・知る・愉しむ 新幹線がわかる事典』、p.20。
    ^ a b 『新幹線 高速鉄道技術のすべて』、p.59。
    ^ a b 『新幹線テクノロジー』、p.162。
    ^ a b c d 『新幹線 高速鉄道技術のすべて』、p.148 - p.149。
    ^ 『鉄道車両メカニズム図鑑』、p.256
    ^ a b c 国松賢四郎「東海道新幹線の電気設備」『電氣學會雜誌』第81巻第879号、電気学会、1961年、2061-2064頁、doi:10.11526/ieejjournal1888.81.2061、ISSN 0020-2878、NAID 130003617667。
    ^ 北陸新幹線「秘話」 日本経済新聞、2020年9月21日閲覧
    ^ 廣田幸嗣著、『電気自動車の本』、日刊工業新聞社、2009年11月25日初版1刷発行、ISBN 9784526063572、152頁
    ^ 変電 - 鉄道建設・運輸施設整備支援機構（2012年1月24日閲覧）
    ^ 井上孝二著『新幹線が一番わかる』技術評論社、2009年2月29日初版第1冊発行、ISBN 978-4774137315
    ^ 全国新幹線鉄道整備法施行規則（昭和45年運輸省令第86号）附則第3項
    ^ JR北など、北海道・東北新幹線の半額切符 日本経済新聞電子版 2018-04-17閲覧
    ^ 出典 : 「東北・上越両新幹線について」大塚滋 「鉄道ファン」No.132 （1972年4月号）
    ^ 新幹線、目前で消えた「東北―東海道」直通計画 日本経済新聞 2014年1月31日
    ^ 『平成21年度版　よくわかる青森県』 pp.113-115 (PDF) - 青森県企画政策部企画調整課
    ^ “【熱血知事ダイアリー】神奈川県とリニア検討委発足へ アクア値下げも合意”. 産経新聞 (産業経済新聞社). (2009年4月7日). オリジナルの2009年4月10日時点におけるアーカイブ。 2014年10月5日閲覧。
    ^ 日本鉄道建設公団東京新幹線建設局・編『上越新幹線工事誌（大宮・新潟間）』
    ^ 会計検査院昭和59年度決算検査報告
    ^ 「東北新幹線工事誌（上野-大宮間）」P112
    ^ “ドボ博 学芸員の部屋008 幻の上越・北陸新幹線新宿駅起点構想について”. 土木学会. 2019年9月29日閲覧。
    ^ 『北海道新幹線の列車名決定について』（PDF）（プレスリリース）北海道旅客鉄道、2014年11月20日。2014年11月20日閲覧。
    ^ 『北海道新幹線の列車名決定について』（PDF）（プレスリリース）東日本旅客鉄道、2014年11月20日。2014年11月20日閲覧。
    ^ 2021年3月ダイヤ改正について - 東日本旅客鉄道高崎支社、2020年12月18日
    ^ 『日本の私鉄5 小田急』 p.115
    ^ a b 『ゼロ戦から夢の超特急 小田急SE車世界新記録誕生秘話』 p.188
    ^ 『新幹線をつくった男 島秀雄物語』 p.160
    ^ 『新幹線をつくった男 島秀雄物語』 pp.160-161
    ^ 『ゼロ戦から夢の超特急 小田急SE車世界新記録誕生秘話』 p.15
    ^ 黎明期の新幹線と中国高速鉄道 平川祐弘 東京大学名誉教授 平川祐弘 国家基本問題研究所
    ^ 恵知仁 (2014年9月24日). “新幹線は「世界四バカ」　根強かった不要論”. 乗りものニュース. 株式会社メディア・ヴァーグ. 2020年3月28日閲覧。
    ^ 加藤一郎「新幹線用高速電動列車と基礎研究」『日本機械学会誌』第66巻第528号、日本機械学会、1963年、125-132頁、doi:10.1299/jsmemag.66.528_125、ISSN 00214728、NAID 110002461550。
    ^ 西尾源太郎「東海道新幹線試作電車の概要」『日本機械学会誌』第66巻第532号、日本機械学会、1963年、688-695頁、doi:10.1299/jsmemag.66.532_688、ISSN 00214728、NAID 110002461868。
    ^ 『読む・知る・愉しむ 新幹線がわかる事典』、p.28。
    ^ 『図解雑学 くわしくわかる新幹線のしくみ』、p.10。
    ^ 宇野俊一ほか編 『日本全史（ジャパン・クロニック）』 講談社、1991年、1128頁。ISBN 4-06-203994-X。
    ^ a b 須田寛『東海道新幹線』p.50
    ^ a b 須田寛『東海道新幹線』p.55
    ^ “新幹線の「安全神話」・・・メンテンスと運行管理がつくりだす＝中国メディア”. サーチナ. (2015年3月4日). オリジナルの2015年3月20日時点におけるアーカイブ。 2015年3月21日閲覧。
    ^ “Safest high-speed rail network”. GUINNESS WORLD RECORDS. 2021年8月16日閲覧。
    ^ 『鉄道ジャーナル』第31巻第8号、鉄道ジャーナル社、1997年8月、104頁。
    ^ のぞみ車内に白煙・異臭　最大3時間以上遅れ - 朝日新聞、2010年3月4日
    ^ 週刊朝日、2004年11月12日。
    ^ “東北新幹線は脱線。高速道にも大きな亀裂...”. ハフポスト日本版 (2022年3月17日). 2022年3月18日閲覧。
    ^ “東北新幹線、脱線しても横転はせず　過去の事故教訓に導入された装置”. 朝日新聞 (朝日新聞社). (2022年3月17日) 2022年3月18日閲覧。
    ^ a b “東北新幹線、月内の全線再開めどたたず　電柱など損傷”. 日本経済新聞 (日本経済新聞社). (2022年3月17日) 2022年3月18日閲覧。
    ^ 東海道・山陽新幹線車内の防犯カメラの増設について (PDF) - 東海旅客鉄道プレスリリース、2015年7月6日
    ^ a b “東海道新幹線、客室内の常時録画開始　放火事件受け”. 朝日新聞デジタル. (2016年2月23日) 2016年2月23日閲覧。
    ^ 『東海道・山陽新幹線の全編成へのAED（自動体外式除細動器）搭載について』（プレスリリース）西日本旅客鉄道、2008年7月28日。
    ^ リニア新幹線、世界最高速度となる時速603 kmを記録 JR東海 マイナビニュース 2015年4月21日
    ^ a b [1]
    ^ http://houston.culturemap.com/newsdetail/08-16-12-full-speed-ahead-for-205-mph-bullet-train-between-houston-and-dallas-2020-set-as-target-date/
    ^ http://www.nbcdfw.com/news/local/Could-High-Speed-Train-Rev-Texas-Economy-362887001.html
    ^ “The Bullet Train That Could Change Everything”. The Texas Tribune. The Texas Tribune. 2015年12月17日閲覧。
    ^ “Learn the facts”. Texas Central Railway. 2015年12月17日閲覧。
    ^ 『英国の都市間高速鉄道計画向け車両の追加受注の内定について』（プレスリリース）日立製作所、2013年7月18日。2014年10月5日閲覧。
    ^ Government gives green light for more state-of-the-art intercity trains - GOV.UK（2013年7月18日）
    ^ “JR東海会長「高速鉄道は技術盗用、安全軽視」に中国高官反論”. サーチナ (2010年4月13日). 2011年1月9日閲覧。
    ^ “中国高速鉄道「独自技術でない」 元幹部、中国紙に暴露”. 朝日新聞. (2011年6月22日). オリジナルの2011年6月23日時点におけるアーカイブ。 2011年1月9日閲覧。
    ^ “中国、新幹線技術盗用は「国家の意思」 米機関が国有企業の分析報告”. MSN産経ニュース. (2011年10月28日) 2011年11月4日閲覧。
    ^ 「死者43人に負傷者は190人以上 鉄道局長ら幹部3人更迭」- MSN産経ニュース（2011年7月24日付）
    ^ 新幹線導入へ調査開始 タイと合意、670キロ、専用軌道建設 産経新聞
    ^ タイ、新幹線採用で日本と合意 台湾に続く２例目 朝日新聞
    ^ タイ高速鉄道・バンコク―チェンマイ間に新幹線導入－両政府で覚書、事業規模は１兆円 日刊工業新聞
    ^ 日本の新幹線2020年タイに開通 テレビ東京
    ^ 日本とタイ、高速鉄道建設で協力覚書締結へ 読売新聞
    ^ 日経産業新聞2007年5月21日4面
    ^ http://www.wsj.com/articles/indian-cabinet-agrees-to-pick-japan-for-high-speed-rail-project-1449744059
    ^ 【ブラジルに新幹線売り込み】日本勢、高速鉄道建設で各国と競合（インターネットアーカイブ） 日経ネット、2009年6月17日
    ^ 【ブラジルに新幹線進出狙う】三井物産、建設に応札へ 共同通信、2009年6月12日
    ^ a b c d 新幹線50年史 p.664
    ^ 平野光芳 (2015年3月22日). “インドネシア大統領:新幹線は優先度低い　インフラ整備で”. 毎日新聞 2015年3月22日閲覧。
    ^ “Indonesia to award fast train contract to China - Japanese embassy official”. Reuters. (2015年9月29日) 2015年10月1日閲覧。
    ^ “Indonesia awards multi-billion-dollar railway project to China over Japan”. ABC. (2015年9月30日) 2015年10月1日閲覧。
    ^ “インドネシア高速鉄道が開業、東南アジア初　中国が援助”. CNN. (2023年10月2日) 2024年1月19日閲覧。
    ^ マレーシア首相、高速鉄道「新幹線も候補」 日本経済新聞
    ^ a b Mitsubishi, favorita para el tren de alta velocidad de Renfe - EL PAÍS - 1988年7月7日作成・2020年10月24日閲覧
    ^ a b Proyecto Fin de Carrera - Estudio crítico sobre las líneas de alta velocidad en España. - Enrique Palacio Gavín - 2014年9月作成・2021年6月10日閲覧
    ^ a b NHK「海外ネットワーク」（特集 世界に売り込め新幹線 激しい競争）2009年8月16日放映
    ^ a b c d “「2024年問題」の切り札に？新幹線で輸送するのは… | NHK | ビジネス特集”. NHKニュース. 日本放送協会 (2023年7月6日). 2023年7月12日閲覧。
    ^ 『平成16年度「鉄道の日」国土交通大臣表彰について』（PDF）（プレスリリース）日本貨物鉄道、2004年10月13日。2014年10月5日閲覧。
    ^ 新幹線利用高速貨物輸送の可能性について (PDF) - 大阪産業大学 波床正敏・田村信弥
    ^ 「幻の貨物新幹線」半世紀残った遺構、来年にも見納めに　東京〜大阪5時間半 - 乗りものニュース、2016年8月14日
    ^ a b c d e 「幻の貨物新幹線 本気だった」『東京新聞』、2013年7月29日、1面。2013年7月29日閲覧。オリジナルの2013年8月1日時点におけるアーカイブ。
    ^ “半世紀残った“謎の高架橋” ひっそりと消える「幻の貨物新幹線」の残骸 大阪・摂津”. MSN産経ニュース. (2014年1月20日) 2014年1月20日閲覧。
    ^ 【高橋団吉著「新幹線を走らせた男 国鉄総裁十河信二物語」について - 長　尚のホームページ 2016年3月24日
    ^ 人口減少と鉄道 石井幸孝著　朝日新書 ISBN 978-4022737601　新幹線物流で日本列島が蘇る p104
    ^ 『新幹線十年史』p.313
    ^ 仁杉巌・石川光男共著『世界一の新幹線』鹿島研究所出版会、1968年、p.157
    ^ 「新幹線鉄道時代を迎えて - 21世紀の鉄道を目指して -」「国鉄線」昭和47年3月号
    ^ 「東北・上越両新幹線について」大塚滋 『鉄道ファン』No.132 （1972年4月号）
    ^ JTBキャンブックス『幻の国鉄車両』 pp. 53
    ^ トレイン・オン・トレイン始動。（上） - ネコ・パブリッシング『鉄道ホビダス』 編集長敬白 2010年2月26日。（インターネットアーカイブ）
    ^ トレイン・オン・トレイン始動。（下） - ネコ・パブリッシング『鉄道ホビダス』 編集長敬白 2010年2月27日。（インターネットアーカイブ）
    ^ 「青函に時速200キロ「貨物新幹線」検討 新幹線は東京へ最速3時間台」『北海道新聞』、2016年1月1日。オリジナルの2026年1月1日時点におけるアーカイブ。
    ^ a b 森創一郎 (2021年8月3日). “廃線危機で浮上､｢貨物新幹線構想｣への熱視線”. 東洋経済オンライン. 2022年11月16日閲覧。
    ^ 「JR西日本、福山通運と貨物輸送 新幹線活用」『日本経済新聞』、2021年7月7日。2021年8月3日閲覧。
    ^ a b c “ズワイガニを新幹線輸送、ＪＲ西 即日販売、広がる貨客混載：東京新聞 TOKYO Web”. 東京新聞 TOKYO Web. 2022年12月4日閲覧。
    ^ a b c d e f g h i “ＪＲ旅客が荷物輸送の事業化に本腰”. 物流の専門紙 カーゴニュース. 2021年1月29日閲覧。
    ^ 日本郵便とＪＲ東日本との連携協定に基づく物流トライアルの実施 - 東日本旅客鉄道・日本郵便、2018年11月5日
    ^ 「獲れたて鮮魚を東京へ、新幹線物流に挑むJR東グループ」 日経ビジネス電子版、2019年6月12日
    ^ 東京駅で「嶽きみ」限定販売　新幹線で21日試験輸送 - 日本経済新聞、2020年8月19日
    ^ 弘前特産トウモロコシ、朝取れ「嶽きみ」東京へ　新幹線で輸送、限定販売 - 河北新報、2020年8月22日
    ^ “旅客列車で輸送した地産品のブランド「FRESH WEST」誕生！ 第一弾は、「活け松葉ガニ」「朝〆お嬢サバ」を即日輸送し販売します：JR西日本”. www.westjr.co.jp. 2022年12月4日閲覧。
    ^ 『九州新幹線を使用した貨客混載事業を開始します』（PDF）（プレスリリース）九州旅客鉄道／佐川急便、2021年5月10日。2021年8月4日閲覧。
    ^ 『九州新幹線荷物輸送サービス「はやっ!便」スタートします!』（PDF）（プレスリリース）九州旅客鉄道、2021年5月10日。2021年8月4日閲覧。
    ^ a b “ＪＲ貨物、30年度までのグループ長期ビジョン発表”. 物流の専門紙 カーゴニュース. 2021年1月29日閲覧。
    ^ “JR貨物グループ 長期ビジョン2030”. JR貨物. 2021年1月30日閲覧。
    ^ “鉄道：今後の鉄道物流のあり方に関する検討会 - 国土交通省”. www.mlit.go.jp. 2022年11月13日閲覧。
    ^ a b c “貨物鉄道は「公的ミッション背負う唯一無二の存在」＝国交省”. カーゴニュース. 2022年11月13日閲覧。
    ^ a b c d 今後の鉄道物流の在り方に関する検討会 (2022年7月28日). “変化し続ける社会の要請に応える貨物鉄道輸送の実現に向けて ～検討会中間とりまとめ～”. 国土交通省鉄道局貨物鉄道政策室. p. 32. 2022年11月13日閲覧。
    ^ JR九州が発表した「全路線の通信簿」から見えること (5/5) - ITmedia ビジネスオンライン、2017年8月4日
    ^ 『「お得なきっぷ」の一部見直しについて』（プレスリリース）東海旅客鉄道、2013年6月28日。2014年10月5日閲覧。
    ^ 台湾高速鉄道｜ アジア&オセアニア｜日本の輸出｜JORSA 日本鉄道システム輸出組合
    ^ “兵庫に4駅集中なぜ？幻の「夜行新幹線」計画”. 神戸新聞 (神戸新聞社). (2012年3月15日). オリジナルの2012年3月17日時点におけるアーカイブ。 2020年4月2日閲覧。
    ^ “オリンピックにあわせて設定「夜行新幹線」　実は過去にも運転されていた”. 鉄道コム (2021年7月2日). 2021年7月30日閲覧。
    ^ 東京 2020 オリンピック期間中における臨時列車の運転について - JR東日本・2021年6月30日
    ^ 東京 2020 オリンピック期間中における臨時列車の運転計画について - JR東日本・2021年7月13日
    ^ 全国新幹線鉄道網を利用した夜行列車の運行可能性に関する検討 (PDF) - 大阪産業大学工学部 波床正敏、井上喜裕
    ^ 東海道・山陽・九州新幹線列車運行図表、2011年3月12日改正、東海旅客鉄道株式会社新幹線鉄道事業本部
    ^ “百田夏菜子【LIVE】ひかり（from 百田夏菜子ソロコンサート『Talk With Me ～シンデレラタイム～』）”. YouTube. 2022年7月12日閲覧。

参考文献

（著者・編者等の五十音・アルファベット順）

    青田孝『ゼロ戦から夢の超特急 小田急SE車世界新記録誕生秘話』交通新聞社、2009年。ISBN 978-4330105093。
    生方良雄、諸河久『日本の私鉄5 小田急』保育社、1981年。0165-508530-7700。
    岡山惇『東北・上越新幹線』中央公論社〈中公新書〉、1985年9月。ISBN 4-12-100777-8。
    川島令三 監修 編『図解雑学 くわしくわかる新幹線のしくみ』ナツメ社、2009年7月。ISBN 9784816346781。
    川島令三、岡田直『鉄道「歴史・地理」なるほど探検ガイド - 大都市圏・新幹線版』PHP研究所、2002年4月。ISBN 4569619886。
    高速鉄道研究会『新幹線 高速鉄道技術のすべて』山海堂、2003年10月。ISBN 4381015487。
    交通協力会 『新幹線50年史』 交通新聞社、2015年。ISBN 978-4330567150。
    佐藤芳彦『新幹線テクノロジー - 0系から800系九州新幹線の高速車両技術』山海堂、2004年3月。ISBN 4381088271。
    須田寬『東海道新幹線』JTBパブリッシング、2000年8月。ISBN 4533035639。
    須田寛『東海道新幹線〈2〉』JTBパブリッシング、2003年12月。ISBN 4533050573。
    高橋団吉『新幹線をつくった男 島秀雄物語』小学館、2000年。ISBN 978-4093410311。
    所澤秀樹『国鉄の戦後がわかる本 上巻』山海堂、2000年2月。ISBN 4381103602。
    所澤秀樹『国鉄の戦後がわかる本 下巻』山海堂、2000年3月。ISBN 438110367X。
    原口隆行『読む・知る・愉しむ 新幹線がわかる事典』日本実業出版社、2005年6月。ISBN 4534039158。
    南谷昌二郎『山陽新幹線』JTBパブリッシング、2005年2月。ISBN 4533058825。
    山之内秀一郎『新幹線がなかったら』東京新聞出版局、1998年12月。ISBN 4808306581。
    山之内秀一郎『東北・上越新幹線』JTBパブリッシング、2002年12月。ISBN 4533045138。
    Christopher P. Hood (2006) (英語). Shinkansen - From Bullet Train to Symbol of Modern Japan. Routledge Contemporary Japan Series. London, UK: Routledge. ISBN 0415320526
    日本国有鉄道建設局新幹線工事課『東北新幹線工事誌－上野・大宮間－』、日本国有鉄道、1986年11月。

外部リンク
ウィキメディア・コモンズには、新幹線に関連するメディアおよびカテゴリがあります。

    JR東日本
        車両図鑑＞新幹線
        新幹線車内設備のご案内
    JR東海
        車両のご案内
        東海道新幹線 開業50周年記念サイト - ウェイバックマシン（2015年3月14日アーカイブ分）
    JR西日本
        新幹線について：JRおでかけネット
        新幹線│車両案内：JRおでかけネット
    JR九州
        JR九州の列車たち 〜JR九州 観光列車・新幹線〜

    表話編歴

高速鉄道

    表話編歴

日本の新幹線

    表話編歴

日本の鉄道史

    表話編歴

日本の旗 日本関連の主要項目
典拠管理データベース: 国立図書館 ウィキデータを編集	

    アメリカ 日本

カテゴリ:

    新幹線新幹線の路線日本の鉄道高速鉄道
"""

In [19]:
question = "新幹線が最初に開通したのはいつですか？"
answer = "新幹線は1964年10月1日に東京オリンピックに合わせて開業しました。最初の路線は東海道新幹線で、東京と大阪を結びました。新幹線は1日に1億人に利用されるほど大規模なサービスとなっています。"


generate_statement_prompt = f"""
後述する質問<question></question>と回答内容<answer></answer>があった時に、回答内容の各文章から 1 つ以上の命題<statements></statements>を抜き出してください。
<question>
{question}
</question>
<answer>
{answer}
</answer>
出力する際は、下記フォーマットに従ってください。
<statements>
  <statement>{{ここに抽出された命題を記す}}</statement>
  <statement>{{ここに抽出された命題を記す}}</statement>
</statements>
"""

In [ ]:
response = invoke_model("", generate_statement_prompt)
pattern = r"<statement>(.*?)</statement>"
response_text = response["content"][0]["text"]
generated_statements = re.findall(pattern, response_text, re.DOTALL)
print("Generated statements: ")
pprint(generated_statements)

In [ ]:
def format_statements(statements: list[str]):
    """命題の中身をプロンプトに入れ込むための関数
    """
    formatted_statements = ""
    for statement in statements:
        formatted_statements += f"<statement>{statement}</statement>\n"
    return formatted_statements.strip()
    
judge_statement_prompt = f"""
下記は文章の評価に使うために必要な背景情報<context></context>です。注意深く読んでください。
<context>
{context}
</context>
与えられた背景情報<context></context>と以下の命題<statements></statements>をみて、各種命題が<context></context>によって裏付けられているかどうかを判断してください。
判断を下す前に、各命題について簡単な説明をつけてください。
出力<respnse></response>は以下のフォーマットに従ってください。指定された形式からは逸脱しないでください。
<statements>
 {format_statements(generated_statements)}
</statements>
<response>
  <statement>
    <thinking>{{この命題が背景情報によって裏付けられているかを判断するための説明}}</thinking>
    <judgement>{{Yes または No で命題が背景情報によって裏付けられているかを判定する}}</judgement>
  </statement>
</response>
"""

judge_statements = invoke_model("", judge_statement_prompt)
pprint(judge_statements)

In [ ]:
judge_statements_text = judge_statements["content"][0]["text"]
pattern = r"<statement>(.*?)</statement>"
statements_list = re.findall(pattern, judge_statements_text, re.DOTALL)
judgement_pattern = r"<judgement>(.*?)</judgement>"
yes_no_list: list[str] = []
for _statement in statements_list:
    yes_no_list += re.findall(judgement_pattern, _statement, re.DOTALL)
sum_score = 0
for _yes_no in yes_no_list:
    if _yes_no == "Yes":
        sum_score += 1.0
faithful_score = sum_score / len(yes_no_list)
print("Faithful Score: ", faithful_score)

## Context Recall

In [ ]:
def format_statements(statements: list[str]):
    """命題の中身をプロンプトに入れ込むための関数
    """
    formatted_statements = ""
    for statement in statements:
        formatted_statements += f"<statement>{statement}</statement>\n"
    return formatted_statements.strip()

ground_truth = "明日の天気は晴れで、予想最高気温は 20 度で最低気温は 15 度です。肌寒いので１枚上に羽織ると良いでしょう。"
context = "明日の天気: 晴れ、最高気温: 20 度"

generate_statement_prompt = f"""
後述する回答内容<answer></answer>があった時に、回答内容の各文章から 1 つ以上の命題<statements></statements>を抜き出してください。
<answer>
{ground_truth}
</answer>
出力する際は、下記フォーマットに従ってください。
<statements>
  <statement>{{ここに抽出された命題を記す}}</statement>
  <statement>{{ここに抽出された命題を記す}}</statement>
</statements>
"""

response = invoke_model("", generate_statement_prompt)
pattern = r"<statement>(.*?)</statement>"
response_text = response["content"][0]["text"]
generated_statements = re.findall(pattern, response_text, re.DOTALL)

# Faithfull で用いたプロンプトとほぼ同様
judge_statement_prompt = f"""
下記は文章の評価に使うために必要な背景情報<context></context>です。注意深く読んでください。
<context>
{context}
</context>
与えられた背景情報<context></context>と以下の命題<statements></statements>をみて、各命題で言及されている内容が<context></context>に含まれているかを判断してください。
判断を下す前に、背景情報<context></context>のどの部分に含まれるかの考察をしてください。
出力<respnse></response>は以下のフォーマットに従ってください。指定された形式からは逸脱しないでください。
<statements>
 {format_statements(generated_statements)}
</statements>
<response>
  <statement>
    <thinking>{{この命題で言及されている情報が背景情報に含まれているかを判断するための考察}}</thinking>
    <judgement>{{Yes または No で命題が背景情報に含まれているかを判定}}</judgement>
  </statement>
</response>
"""
judge_statements = invoke_model("", judge_statement_prompt)
judge_statements_text = judge_statements["content"][0]["text"]
pattern = r"<statement>(.*?)</statement>"
statements_list = re.findall(pattern, judge_statements_text, re.DOTALL)
judgement_pattern = r"<judgement>(.*?)</judgement>"
yes_no_list: list[str] = []
for _statement in statements_list:
    yes_no_list += re.findall(judgement_pattern, _statement, re.DOTALL)
sum_score = 0
for _yes_no in yes_no_list:
    if _yes_no == "Yes":
        sum_score += 1.0
context_recall_score = sum_score / len(yes_no_list)
print("Generated Statements from Ground Truth:", generated_statements)
print(yes_no_list)
print("Context Recall: ", context_recall_score)

In [ ]:
print(generated_statements)
print(judge_statements_text)